# Lab 5: Microsoft Agent Framework (MAF) - 워크플로우 패턴 및 오케스트레이션

## 개요 (Overview)

이 노트북에서는 **Azure AI Foundry Agent**와 **Microsoft Agent Framework (MAF)**를 결합하여 다양한 워크플로우 패턴과 멀티-에이전트 오케스트레이션을 실습합니다.

### 핵심 개념

**🤖 AI Agents (개별 에이전트) - Azure AI Foundry Agent 사용**
- Azure AI Foundry의 Agent API를 사용하여 에이전트 정의
- LLM을 사용하여 사용자 입력 처리
- 도구 및 MCP 서버 호출을 통한 작업 수행
- Thread 기반의 상태 관리
- 멀티턴 대화 지원

**⚡ Workflows (워크플로우) - Microsoft Agent Framework (MAF) 사용**
- MAF의 그래프 기반 워크플로우로 복잡한 오케스트레이션 정의
- Foundry Agent로 정의된 여러 에이전트와 함수의 조합
- 타입 안전성 및 상태 관리
- 조건부 라우팅, 병렬 처리, 동적 실행

### 아키텍처: Foundry Agent + MAF Workflow

```
┌──────────────────────────────────────────────────────────────────┐
│              MAF Workflow Orchestration Layer                    │
│                                                                  │
│  ┌────────────────┐  ┌────────────────┐  ┌────────────────┐   │
│  │  Sequential    │  │  Concurrent    │  │  Handoff       │   │
│  │  Workflow      │  │  Workflow      │  │  Workflow      │   │
│  └───────┬────────┘  └───────┬────────┘  └───────┬────────┘   │
│          │                   │                    │            │
└──────────┼───────────────────┼────────────────────┼────────────┘
           │                   │                    │
┌──────────▼───────────────────▼────────────────────▼────────────┐
│           Azure AI Foundry Agents (Agent Layer)                │
│                                                                 │
│  ┌─────────────────┐  ┌─────────────────┐  ┌────────────────┐ │
│  │  Validator      │  │  Transformer    │  │  Summarizer    │ │
│  │  Agent          │  │  Agent          │  │  Agent         │ │
│  │  (Foundry)      │  │  (Foundry)      │  │  (Foundry)     │ │
│  └─────────────────┘  └─────────────────┘  └────────────────┘ │
│                                                                 │
│  ✅ Thread-based State Management                              │
│  ✅ LLM Integration (GPT-4, GPT-4o, etc.)                      │
│  ✅ Tool/MCP Server Integration                                │
└─────────────────────────────────────────────────────────────────┘
```

### 학습할 오케스트레이션 패턴 (Orchestration Patterns)

MAF를 사용하여 구현하는 워크플로우 패턴:

1. **Sequential Pattern (2번 실습)** - 순차 실행
   - 한 에이전트(Foundry Agent)의 결과를 다음 에이전트에 전달
   - 파이프라인, 다단계 처리에 적합

2. **Concurrent Pattern (3번 실습)** - 동시 실행 (병렬)
   - 모든 에이전트에 작업을 동시에 브로드캐스트
   - 병렬 분석, 앙상블 의사결정에 적합

3. **Conditional Pattern (4번 실습)** - 조건 분기
   - 입력 조건에 따라 다른 전문가 에이전트로 라우팅
   - 의도 분류, 맞춤형 응답에 적합

4. **Loop Pattern (5번 실습)** - 반복 개선
   - 조건이 만족될 때까지 피드백 루프 실행
   - 품질 개선, 반복적 최적화에 적합

5. **Error Handling Pattern (6번 실습)** - 오류 처리 및 복구
   - 실패 감지, 대안 제시, 자동 복구
   - 안정성이 중요한 예약 시스템에 적합

6. **Handoff Pattern (7번 실습)** - 동적 제어 이전
   - 복잡도에 따라 에이전트 간 제어 동적 이전
   - 에스컬레이션, 전문가 배정에 적합

### 선행 요구사항 (Prerequisites)

- ✅ Python 3.10+
- ✅ Azure AI Foundry Project 설정
- ✅ Microsoft Agent Framework (MAF) 설치
- ✅ Azure OpenAI 또는 OpenAI API 설정
- ✅ Azure OpenAI 또는 OpenAI API 설정
- ✅ 기본 Python asyncio 이해

---

## 📑 목차 (Table of Contents)

### 기본 설정
1. **필수 라이브러리 및 환경 설정**
2. **Azure AI Foundry Agent Client 초기화**

### 워크플로우 패턴 실습
2. **Sequential Pattern** - 순차 실행 패턴
3. **Concurrent Pattern** - 병렬 실행 패턴  
4. **Conditional Pattern** - 조건 분기 패턴
5. **Loop Pattern** - 반복 개선 패턴
6. **Error Handling Pattern** - 오류 처리 패턴
7. **Handoff Pattern** - 동적 라우팅 패턴

### 종합 정리
- **워크플로우 패턴 종합 비교**
- **실습별 핵심 개념 정리**
- **베스트 프랙티스 및 최적화**
- **프로덕션 가이드**

---

## 1. 필수 라이브러리 및 환경 설정 (Import & Setup)

In [ ]:
# ========================================================================
# 📦 필수 라이브러리 Import (Required Libraries)
# ========================================================================

import os
import sys
import time
import asyncio
import logging
from datetime import datetime
from typing import Dict, Any, List, Optional
from dataclasses import dataclass

# Azure AI Foundry
from azure.ai.projects import AIProjectClient
from azure.identity import AzureCliCredential, ChainedTokenCredential, ManagedIdentityCredential
from azure.identity.aio import (
    AzureCliCredential as AsyncAzureCliCredential,
    ManagedIdentityCredential as AsyncManagedIdentityCredential,
    ChainedTokenCredential as AsyncChainedTokenCredential
)

# Microsoft Agent Framework (MAF) - WorkflowBuilder
from agent_framework.azure import AzureAIAgentClient
from agent_framework import WorkflowBuilder, WorkflowContext, executor

# Logging 설정 - 간결한 출력
logging.basicConfig(
    level=logging.WARNING,
    format='%(levelname)s: %(message)s'
)
logger = logging.getLogger(__name__)

# Azure SDK 로그 레벨 조정
logging.getLogger('azure').setLevel(logging.ERROR)
logging.getLogger('azure.core.pipeline.policies.http_logging_policy').setLevel(logging.ERROR)
logging.getLogger('azure.identity').setLevel(logging.ERROR)

print("✅ 필수 라이브러리 Import 완료")
print(f"   - Python 버전: {sys.version.split()[0]}")
print(f"   - Azure AI Projects SDK 로드 완료")
print(f"   - Microsoft Agent Framework (MAF) WorkflowBuilder 로드 완료")
print(f"   - Asyncio 준비 완료")

In [ ]:
# ========================================================================
# 🔐 Azure CLI 로그인 확인 (Check Azure CLI Login)
# ========================================================================

import subprocess
import json

def check_azure_login():
    """Azure CLI 로그인 상태 확인 및 상세 정보 반환"""
    try:
        result = subprocess.run(
            ["az", "account", "show"],
            capture_output=True,
            text=True,
            timeout=10
        )
        if result.returncode == 0:
            account_info = json.loads(result.stdout)
            return True, account_info
        return False, None
    except Exception as e:
        return False, str(e)

print("\n" + "="*70)
print("🔐 Azure CLI 인증 상태 검증")
print("="*70)

is_logged_in, account_info = check_azure_login()

if is_logged_in:
    print("\n✅ Azure CLI 인증 성공\n")
    print(f"📋 계정 정보:")
    print(f"   ├─ 구독 이름: {account_info.get('name', 'N/A')}")
    print(f"   ├─ 구독 ID: {account_info.get('id', 'N/A')}")
    print(f"   ├─ 테넌트 ID: {account_info.get('tenantId', 'N/A')}")
    print(f"   ├─ 사용자: {account_info.get('user', {}).get('name', 'N/A')}")
    print(f"   └─ 상태: {account_info.get('state', 'N/A')}")
    
    print(f"\n🎯 Foundry Agent 사용 준비 완료")
    print(f"   Azure AI Project 리소스에 접근 가능합니다.\n")
else:
    print("\n❌ Azure CLI 인증 실패\n")
    print(f"⚠️  문제:")
    print(f"   {account_info if account_info else '로그인되지 않음'}\n")
    print(f"🔧 해결 방법:")
    print(f"   1. 터미널에서 다음 명령어 실행:")
    print(f"      $ az login")
    print(f"   2. 브라우저에서 인증 완료")
    print(f"   3. 이 셀을 다시 실행\n")

print("="*70 + "\n")

## 🔧 Azure AI Foundry Client 초기화

이 섹션에서는 **Azure AI Foundry Agent**와 통신하기 위한 클라이언트를 초기화합니다.

### 초기화 과정:

**1️⃣ 설정 파일 로드**
- `config.json` 파일에서 프로젝트 연결 문자열 및 모델 정보 읽기
- 환경 변수에서도 설정 가능 (환경 변수 우선)

**2️⃣ 연결 문자열 파싱**
- Azure AI Project 엔드포인트 URL 추출
- 연결 문자열 형식: `https://...;subscription_id=...;resource_group=...`
- 세미콜론(`;`) 앞부분만 엔드포인트로 사용

**3️⃣ 인증 설정**
- `AzureCliCredential` 사용 (Azure CLI 로그인 기반)
- `az login` 명령으로 사전 인증 필요

**4️⃣ AIProjectClient 생성**
- 엔드포인트와 credential을 사용하여 클라이언트 생성
- Foundry Agent API 호출을 위한 기본 클라이언트

**5️⃣ 검증**
- 연결 성공 여부 확인
- 오류 발생 시 해결 방법 안내

### 필수 조건:
- ✅ Azure CLI 설치 및 로그인 (`az login`)
- ✅ `config.json` 파일에 `project_connection_string` 설정
- ✅ Azure AI Project 리소스 생성 및 접근 권한

### 설정 정보:
- **Connection String**: Azure AI Project 엔드포인트 URL
- **Model Deployment**: 사용할 LLM 모델 이름 (예: `gpt-4o`)

In [ ]:
# ========================================================================
# 🔧 MAF Agent Client 초기화 (Initialize MAF Agent Client)
# ========================================================================

print("\n" + "="*70)
print("🔧 Microsoft Agent Framework (MAF) Client 초기화")
print("="*70)

# config.json에서 설정 로드
import json
config_path = "config.json"
project_endpoint = None
model_name = None  # config.json에서 로드

if os.path.exists(config_path):
    with open(config_path, 'r') as f:
        config = json.load(f)
        project_connection_string = config.get("project_connection_string")
        
        # Connection string에서 엔드포인트만 추출 (첫 번째 ; 이전 부분)
        if project_connection_string and ';' in project_connection_string:
            project_endpoint = project_connection_string.split(';')[0]
        else:
            project_endpoint = project_connection_string
        
        # config.json에서 model_deployment_name 로드
        model_name = config.get("model_deployment_name")
        
        if not model_name:
            print(f"\n⚠️  config.json에 'model_deployment_name' 설정이 없습니다")
            print(f"   기본값 'gpt-4o'를 사용합니다")
            model_name = "gpt-4o"
        
        print(f"\n✅ config.json 로드 성공")
else:
    print(f"\n⚠️  config.json 파일을 찾을 수 없습니다")
    print(f"   기본값 'gpt-4o'를 사용합니다")
    model_name = "gpt-4o"

# 환경 변수로도 설정 가능 (우선순위: 환경 변수 > config.json)
if os.getenv("AZURE_AI_PROJECT_ENDPOINT"):
    project_endpoint = os.getenv("AZURE_AI_PROJECT_ENDPOINT")

if os.getenv("AZURE_AI_MODEL_DEPLOYMENT_NAME"):
    model_name = os.getenv("AZURE_AI_MODEL_DEPLOYMENT_NAME")

print(f"\n📋 설정 정보:")
print(f"   ├─ Project Endpoint: {project_endpoint[:50] + '...' if project_endpoint else '❌ 미설정'}")
print(f"   └─ Model Deployment: {model_name}")

# MAF Agent Client 초기화
agent_client = None

if not project_endpoint:
    print(f"\n⚠️  프로젝트 엔드포인트 미설정")
    print(f"   config.json 파일을 확인하거나 환경 변수를 설정하세요.")
    print(f"   예: export AZURE_AI_PROJECT_ENDPOINT='<your-endpoint-url>'")
else:
    try:
        # Azure CLI 인증 사용 (비동기 credential 필요)
        async_credential = AsyncChainedTokenCredential(
            AsyncManagedIdentityCredential(),
            AsyncAzureCliCredential()
        )
        
        # MAF Agent Client 생성
        agent_client = AzureAIAgentClient(
            project_endpoint=project_endpoint,
            model_deployment_name=model_name,
            async_credential=async_credential
        )
        
        print(f"\n✅ MAF Agent Client 초기화 성공")
        print(f"   - 인증: Azure CLI / Managed Identity (Async)")
        print(f"   - Endpoint: {project_endpoint}")
        print(f"   - Model: {model_name}")
        print(f"   - WorkflowBuilder 사용 가능")
        
    except Exception as e:
        print(f"\n❌ MAF Agent Client 초기화 실패")
        print(f"   오류: {type(e).__name__}: {str(e)}")
        print(f"\n💡 해결 방법:")
        print(f"   1. az login 실행")
        print(f"   2. config.json 파일 확인")
        print(f"   3. Azure AI Project 리소스 확인")
        print(f"   4. agent_framework 패키지 설치 확인: pip install agent-framework[azure-ai]")
        agent_client = None

print("="*70 + "\n")

## 워크플로우 패턴의 기본 개념 (Workflow Pattern Concepts)

### 1️⃣ Sequential Pattern (순차 패턴)
```
Input → Task 1 → Task 2 → Task 3 → Output
        (완료 후)  (완료 후)
```
- **특징**: 각 작업이 순서대로 실행, 이전 작업의 결과를 다음 작업이 사용
- **사용처**: 데이터 변환 파이프라인, 다단계 검증
- **장점**: 직관적, 구현 간단
- **단점**: 느린 실행 (병렬화 불가능)

### 2️⃣ Parallel Pattern (병렬 패턴)
```
      ┌─ Task 1 ┐
Input ┤─ Task 2 ├─ Aggregation → Output
      └─ Task 3 ┘
```
- **특징**: 여러 작업을 동시에 실행
- **사용처**: 독립적인 정보 수집, 멀티 에이전트 쿼리
- **장점**: 빠른 실행, 리소스 활용 효율
- **단점**: 동기화 관리 필요

### 3️⃣ Conditional Branching Pattern (조건부 분기)
```
       ┌─ (조건 A) → Path 1
Input ─┤─ (조건 B) → Path 2
       └─ (기본값) → Path 3
            ↓
         Output
```
- **특징**: 조건에 따라 다른 경로로 실행
- **사용처**: 사용자 의도 분류, 동적 라우팅
- **장점**: 유연한 제어
- **단점**: 복잡도 증가

### 4️⃣ Loop Pattern (루프 패턴)
```
Input → [반복 조건 확인]
         ├─ (계속) → Task → 반복 카운터 증가
         └─ (종료) → Output
```
- **특징**: 조건을 만족할 때까지 반복 실행
- **사용처**: 배치 처리, 데이터 정제
- **장점**: 동적 반복 가능
- **단점**: 무한 루프 주의

### 5️⃣ Handoff Pattern (핸드오프 패턴)
```
Input → Agent A → [핸드오프 조건]
                    ├─ (전문가 필요) → Agent B (Specialist)
                    ├─ (승인 필요) → Agent C (Approver)
                    └─ (완료 가능) → Output
```
- **특징**: 컨텍스트나 조건에 따라 다른 에이전트로 동적 제어 이전
- **사용처**: 고객 지원 에스컬레이션, 승인 워크플로우, 전문가 상담
- **장점**: 유연한 제어 이전, 전문성 활용
- **단점**: 에이전트 간 컨텍스트 전달 관리 필요

### 6️⃣ Error Handling & Retry Pattern (오류 처리)
```
Input → Try Task → [오류 발생?]
                   ├─ (Yes) → Retry (exp backoff) → 최대 횟수?
                   │          ├─ (미도달) → 재시도
                   │          └─ (도달) → Fallback
                   └─ (No) → Output
```
- **특징**: 오류 발생 시 재시도 및 대체 전략
- **사용처**: API 호출, 네트워크 작업
- **장점**: 안정성 증가
- **단점**: 구현 복잡도

## 🛠️ MAF WorkflowBuilder 유틸리티

이 섹션에서는 MAF의 **WorkflowBuilder 패턴**을 사용하기 위한 핵심 컴포넌트를 정의합니다.

### MAF WorkflowBuilder 개념

**WorkflowBuilder**는 MAF의 핵심 오케스트레이션 도구입니다:

**주요 구성 요소:**
- ✅ **@executor 데코레이터**: Workflow의 각 노드(단계)를 정의
- ✅ **WorkflowContext**: 노드 간 데이터 전달 및 상태 관리
- ✅ **WorkflowBuilder**: 노드와 엣지를 연결하여 실행 그래프 구성
- ✅ **Agent**: 각 노드에서 실제 LLM 호출을 수행하는 Agent

**실행 흐름:**
```
1. Agent 생성 → 2. Executor 노드 정의 → 3. WorkflowBuilder로 그래프 구성
                   ↓
4. Workflow 실행 → 5. Context를 통해 데이터 전달 → 6. 결과 수집
```

### Message 타입 정의

Workflow에서 사용할 메시지 타입을 dataclass로 정의합니다.

In [ ]:
# ========================================================================
# 📦 MAF Message Types (Workflow 데이터 전달용)
# ========================================================================

@dataclass
class TravelRequest:
    """여행 요청 메시지"""
    destination: str
    days: int
    user_query: str
    itinerary: Optional[str] = None
    local_insights: Optional[str] = None
    final_plan: Optional[str] = None

# ========================================================================
# 🤖 Agents 생성 (각 단계별 전문가 Agent)
# ========================================================================

def create_travel_agents():
    """여행 계획을 위한 3개의 전문가 Agent 생성"""
    
    if not agent_client:
        raise ValueError("❌ MAF Agent client가 초기화되지 않았습니다.")
    
    print("\n" + "="*70)
    print("🤖 여행 전문가 Agents 생성 중...")
    print("="*70)
    
    # Agent 1: Travel Planner (기본 일정 작성)
    travel_planner = agent_client.create_agent(
        name="TravelPlanner",
        instructions=(
            "당신은 전문 여행 플래너입니다.\n\n"
            "역할:\n"
            "- 사용자의 여행 요청을 분석하여 상세한 일별 일정 작성\n"
            "- 각 날짜별 주요 방문지 및 활동 제안\n"
            "- 시간대별 세부 계획 (아침/점심/저녁) 포함\n"
            "- 각 장소의 특징 및 볼거리 설명\n"
            "- 이동 시간 및 교통 수단 안내\n\n"
            "스타일: 구체적이고 실행 가능한 일정, 모든 응답은 한글로 작성"
        )
    )
    
    # Agent 2: Local Expert (현지 인사이트 추가)
    local_expert = agent_client.create_agent(
        name="LocalExpert",
        instructions=(
            "당신은 현지 여행 전문가입니다.\n\n"
            "역할:\n"
            "- 기존 여행 일정을 검토하고 현지인만 아는 특별한 장소 추가\n"
            "- 관광객이 잘 모르는 숨겨진 명소 추천\n"
            "- 현지인이 자주 가는 맛집이나 카페 소개\n"
            "- 특별한 현지 문화 체험 기회 제안\n"
            "- 각 장소에서의 실용적인 팁 제공\n\n"
            "스타일: 진정성있고 흥미로운 현지 경험 중심, 모든 응답은 한글로 작성"
        )
    )
    
    # Agent 3: Travel Summarizer (최종 통합 계획서)
    travel_summarizer = agent_client.create_agent(
        name="TravelSummarizer",
        instructions=(
            "당신은 여행 계획 요약 전문가입니다.\n\n"
            "역할:\n"
            "- 모든 제안과 조언을 종합하여 완전한 최종 여행 계획 작성\n"
            "- 일별 상세 일정을 일관되게 통합\n"
            "- 실용적인 여행 팁 (교통, 언어, 에티켓) 포함\n"
            "- 주의사항 및 안전 정보 추가\n"
            "- 예산 가이드라인 제시\n"
            "- 필수 예약 정보 정리\n\n"
            "스타일: 상세하고 완전하며 실용적인 계획서, 모든 응답은 한글로 작성"
        )
    )
    
    print(f"✅ Agent 1: {travel_planner.name} (Travel Planner) 생성 완료")
    print(f"✅ Agent 2: {local_expert.name} (Local Expert) 생성 완료")
    print(f"✅ Agent 3: {travel_summarizer.name} (Travel Summarizer) 생성 완료")
    print("="*70 + "\n")
    
    return travel_planner, local_expert, travel_summarizer

# Agents 생성
travel_planner, local_expert, travel_summarizer = create_travel_agents()

## 2. Sequential Workflow Pattern (순차 워크플로우)

### 🔧 MAF WorkflowBuilder를 사용한 Sequential Pattern 구현

In [ ]:
"""
Sequential Pattern - MAF WorkflowBuilder 사용
- @executor 데코레이터로 각 단계(노드) 정의
- WorkflowBuilder로 노드 연결 및 실행 그래프 구성
- WorkflowContext를 통해 단계 간 데이터 전달
- Azure AI Foundry Agent로 각 단계에서 LLM 호출
"""

# ========================================================================
# Step 1: Executor 노드 정의 (각 단계를 @executor로 정의)
# ========================================================================

@executor(id="travel_planner")
async def travel_planner_node(msg: TravelRequest, ctx: WorkflowContext[TravelRequest]) -> None:
    """Step 1: Travel Planner - 기본 일정 작성"""
    print(f"\n🗺️  Step 1: Travel Planner - 기본 일정 작성 중...")
    print(f"   입력: {msg.user_query}")
    
    # Travel Planner Agent로 기본 일정 생성
    query = f"""다음 여행 요청에 대한 상세한 일정을 작성하세요:

요청: {msg.user_query}

다음을 포함한 일별 일정을 작성하세요:
- 각 날짜별 주요 방문지 및 활동
- 아침, 점심, 저녁 시간대별 세부 계획
- 각 장소의 특징 및 볼거리
- 이동 시간 및 교통 수단
- 추천 식당 및 현지 음식

구체적이고 실행 가능한 일정으로 작성하세요."""
    
    # Agent 실행 (새로운 thread 생성)
    thread = travel_planner.get_new_thread()
    result = await travel_planner.run(query, thread=thread)
    response = result.text if hasattr(result, 'text') else str(result)
    
    # Context에 결과 저장하고 다음 단계로 전달
    msg.itinerary = response
    await ctx.send_message(msg, target_id="local_expert")
    print(f"✅ Step 1 완료: 기본 일정 작성 완료")


@executor(id="local_expert")
async def local_expert_node(msg: TravelRequest, ctx: WorkflowContext[TravelRequest]) -> None:
    """Step 2: Local Expert - 현지 인사이트 추가"""
    print(f"\n🏛️  Step 2: Local Expert - 현지 경험 추가 중...")
    
    # Local Expert Agent로 현지 정보 추가
    query = f"""다음 여행 일정을 검토하고 현지인만 아는 특별한 장소나 활동을 추가하세요:

기존 일정:
{msg.itinerary}

다음을 추가하세요:
- 관광객이 잘 모르는 숨겨진 명소
- 현지인이 자주 가는 맛집이나 카페
- 특별한 현지 문화 체험 기회
- 각 장소에서의 실용적인 팁

일정을 더 풍부하고 진정성있게 만드세요."""
    
    # Agent 실행
    thread = local_expert.get_new_thread()
    result = await local_expert.run(query, thread=thread)
    response = result.text if hasattr(result, 'text') else str(result)
    
    # Context에 결과 저장하고 다음 단계로 전달
    msg.local_insights = response
    await ctx.send_message(msg, target_id="travel_summarizer")
    print(f"✅ Step 2 완료: 현지 경험 추가 완료")


@executor(id="travel_summarizer")
async def travel_summarizer_node(msg: TravelRequest, ctx: WorkflowContext[TravelRequest]) -> None:
    """Step 3: Travel Summarizer - 최종 통합 계획서 작성"""
    print(f"\n📋 Step 3: Travel Summarizer - 최종 계획서 작성 중...")
    
    # Travel Summarizer Agent로 최종 계획 통합
    query = f"""다음 모든 정보를 통합하여 완벽한 최종 여행 계획서를 작성하세요:

원본 요청: {msg.user_query}

기본 일정:
{msg.itinerary}

현지 인사이트:
{msg.local_insights}

최종 계획서에 다음을 포함하세요:
1. 완전히 통합된 일별 상세 일정
2. 실용적인 여행 팁 (교통, 언어, 에티켓)
3. 주의사항 및 안전 정보
4. 예산 가이드라인
5. 필수 예약 정보

모든 정보가 일관되고 완전하게 통합된 최종 여행 계획을 작성하세요."""
    
    # Agent 실행
    thread = travel_summarizer.get_new_thread()
    result = await travel_summarizer.run(query, thread=thread)
    response = result.text if hasattr(result, 'text') else str(result)
    
    # 최종 결과를 Context에 저장하고 출력
    msg.final_plan = response
    await ctx.yield_output(msg)
    print(f"✅ Step 3 완료: 최종 계획서 작성 완료")


# ========================================================================
# Step 2: WorkflowBuilder로 노드 연결 및 실행 그래프 구성
# ========================================================================

sequential_workflow = (
    WorkflowBuilder()
    .set_start_executor(travel_planner_node)          # 시작 노드 설정
    .add_edge(travel_planner_node, local_expert_node)  # Step 1 → Step 2
    .add_edge(local_expert_node, travel_summarizer_node)  # Step 2 → Step 3
    .build()
)

print("\n" + "="*70)
print("✅ Sequential Workflow 그래프 구성 완료")
print("="*70)
print("📊 Workflow 구조:")
print("   travel_planner_node (Step 1)")
print("         ↓")
print("   local_expert_node (Step 2)")
print("         ↓")
print("   travel_summarizer_node (Step 3)")
print("="*70 + "\n")


# ========================================================================
# Step 3: Workflow 실행
# ========================================================================

async def run_sequential_workflow():
    """Sequential Workflow 실행 함수"""
    print("\n" + "="*70)
    print("🚀 Sequential Workflow 실행 시작 (MAF WorkflowBuilder)")
    print("="*70)

    # 입력 메시지 생성
    user_query = "런던 5일 여행 계획을 세워주세요."
    travel_request = TravelRequest(
        destination="런던",
        days=5,
        user_query=user_query
    )

    # Workflow 실행 (run_stream 사용 - async generator)
    outputs = []
    async for event in sequential_workflow.run_stream(travel_request):
        # 이벤트에서 output 추출
        if hasattr(event, 'output') and event.output is not None:
            outputs.append(event.output)
            print(f"📤 이벤트 수신: {type(event.output).__name__}")
        elif hasattr(event, 'data') and event.data is not None:
            outputs.append(event.data)
            print(f"📤 이벤트 수신: {type(event.data).__name__}")

    # 최종 결과 추출 (마지막 output이 최종 TravelRequest)
    if outputs:
        final_result = outputs[-1]
    else:
        raise ValueError("❌ Workflow에서 출력을 받지 못했습니다.")

    print("\n" + "="*70)
    print("📊 Sequential Pattern 결과 (MAF WorkflowBuilder)")
    print("="*70)
    print(f"\n📝 여행 요청: {final_result.user_query}\n")
    print(f"\n{'='*70}")
    print(f"🗺️  Step 1 - Travel Planner (기본 일정):")
    print(f"{'='*70}")
    print(f"{final_result.itinerary}\n")
    print(f"\n{'='*70}")
    print(f"🏛️  Step 2 - Local Expert (현지 인사이트):")
    print(f"{'='*70}")
    print(f"{final_result.local_insights}\n")
    print(f"\n{'='*70}")
    print(f"📋 Step 3 - Travel Summarizer (최종 통합 계획서):")
    print(f"{'='*70}")
    print(f"{final_result.final_plan}")
    print(f"\n{'='*70}")
    
    return final_result

# Workflow 실행
final_result = await run_sequential_workflow()

**Sequential Pattern 분석 (MAF WorkflowBuilder 사용)**

### ✅ 핵심 구성 요소

**1. Message Type (데이터 전달)**
- `TravelRequest` dataclass로 workflow 전체에서 사용할 데이터 구조 정의
- 각 노드가 동일한 메시지 객체를 수정하며 전달

**2. Executor 노드 (@executor 데코레이터)**
- `@executor(id="travel_planner")`: Step 1 - 기본 일정 작성
- `@executor(id="local_expert")`: Step 2 - 현지 인사이트 추가  
- `@executor(id="travel_summarizer")`: Step 3 - 최종 계획서 통합

**3. WorkflowBuilder (실행 그래프 구성)**
```python
workflow = (
    WorkflowBuilder()
    .set_start_executor(travel_planner_node)           # 시작점
    .add_edge(travel_planner_node, local_expert_node)  # Step 1 → 2
    .add_edge(local_expert_node, travel_summarizer_node)  # Step 2 → 3
    .build()
)
```

**4. WorkflowContext (노드 간 제어 흐름)**
- `await ctx.handoff_to(next_node, msg)`: 다음 노드로 제어 이전
- `await ctx.yield_output(msg)`: 최종 결과 출력

### ✅ 실행 흐름

```
TravelRequest 생성
    ↓
travel_planner_node (Foundry Agent: travel_planner)
    → msg.itinerary 생성
    → ctx.handoff_to(local_expert_node)
    ↓
local_expert_node (Foundry Agent: local_expert)
    → msg.local_insights 생성
    → ctx.handoff_to(travel_summarizer_node)
    ↓
travel_summarizer_node (Foundry Agent: travel_summarizer)
    → msg.final_plan 생성
    → ctx.yield_output(msg)
    ↓
최종 결과 수집
```

### ✅ MAF WorkflowBuilder 장점

1. **명확한 실행 그래프**: 시각적으로 노드 연결 구조 파악 가능
2. **타입 안전성**: dataclass로 메시지 구조 명확히 정의
3. **유연한 제어 흐름**: handoff_to로 동적 라우팅 가능
4. **상태 관리**: WorkflowContext가 실행 상태 추적
5. **재사용성**: 각 executor 노드를 다른 workflow에서도 재사용 가능

### ✅ 사용 사례
- 다단계 데이터 변환 파이프라인
- 문서 생성 워크플로우 (초안 → 검토 → 최종본)
- 복잡한 요청 처리 (분석 → 계획 → 실행)
- 점진적 개선이 필요한 작업

## 3. Concurrent Workflow Pattern (동시 실행 워크플로우)

In [ ]:
"""
Concurrent Pattern - MAF WorkflowBuilder 사용
- @executor 데코레이터로 각 전문가 노드 정의
- WorkflowBuilder로 병렬 실행 그래프 구성
- WorkflowContext를 통해 데이터 전달
- 여러 전문가가 동시에 독립적으로 분석
"""

# ========================================================================
# Message Type 정의 (Concurrent Pattern용)
# ========================================================================

@dataclass
class TravelAnalysisRequest:
    """여행 분석 요청 메시지 (병렬 처리용)"""
    destination: str
    user_query: str
    culture_history: Optional[str] = None
    food_dining: Optional[str] = None
    practical_tips: Optional[str] = None
    final_summary: Optional[str] = None

# ========================================================================
# Concurrent Pattern Agents 생성
# ========================================================================

def create_concurrent_agents():
    """병렬 분석을 위한 3개의 전문가 Agent 생성"""
    
    if not agent_client:
        raise ValueError("❌ MAF Agent client가 초기화되지 않았습니다.")
    
    print("\n" + "="*70)
    print("🤖 병렬 분석 전문가 Agents 생성 중...")
    print("="*70)
    
    # Agent 1: Culture & History Expert
    culture_expert = agent_client.create_agent(
        name="CultureHistoryExpert",
        instructions=(
            "당신은 문화 및 역사 전문가입니다.\n\n"
            "역할:\n"
            "- 여행지의 문화적, 역사적 가치를 깊이있게 분석\n"
            "- 꼭 방문해야 할 역사적 명소 추천 (박물관, 유적지, 기념관)\n"
            "- 각 장소의 역사적 의미와 배경 설명\n"
            "- 문화 체험 활동 제안 (전통 공연, 축제, 워크샵)\n"
            "- 현지 에티켓 및 문화적 주의사항 안내\n"
            "- 추천 방문 시간대 및 팁 제공\n\n"
            "스타일: 깊이있고 교육적인 문화 경험 중심, 모든 응답은 한글로 작성"
        )
    )
    
    # Agent 2: Food & Dining Expert
    food_expert = agent_client.create_agent(
        name="FoodDiningExpert",
        instructions=(
            "당신은 미식 전문가입니다.\n\n"
            "역할:\n"
            "- 여행지의 음식 문화와 식도락 경험 소개\n"
            "- 꼭 먹어봐야 할 현지 음식 및 요리 추천\n"
            "- 다양한 맛집 추천 (미슐랭, 현지인 맛집, 길거리 음식)\n"
            "- 각 음식의 특징과 유래 설명\n"
            "- 식사 예산 가이드 제공 (저가/중가/고가)\n"
            "- 채식주의자/알러지 대응 정보 안내\n"
            "- 예약 필요 여부 및 운영 시간 정보\n\n"
            "스타일: 진정한 미식 경험 중심, 모든 응답은 한글로 작성"
        )
    )
    
    # Agent 3: Practical Travel Expert
    practical_expert = agent_client.create_agent(
        name="PracticalTravelExpert",
        instructions=(
            "당신은 실용적인 여행 전문가입니다.\n\n"
            "역할:\n"
            "- 여행자가 꼭 알아야 할 실용 정보 제공\n"
            "- 교통 정보 (공항-시내 이동, 대중교통, 교통카드)\n"
            "- 숙박 정보 (지역별 추천, 가격대, 특징)\n"
            "- 예산 가이드 (일일 예산: 숙박/교통/식사/관광)\n"
            "- 통신 정보 (SIM카드, WiFi, 추천 앱)\n"
            "- 안전 수칙 및 긴급 연락처\n"
            "- 쇼핑 정보 (추천 지역, 면세점, 기념품)\n"
            "- 날씨 및 복장 가이드\n\n"
            "스타일: 실질적이고 유용한 정보 중심, 모든 응답은 한글로 작성"
        )
    )
    
    # Agent 4: Final Summarizer (병렬 분석 결과 통합)
    final_summarizer = agent_client.create_agent(
        name="FinalSummarizer",
        instructions=(
            "당신은 종합 여행 계획 작성 전문가입니다.\n\n"
            "역할:\n"
            "- 여러 전문가의 분석 결과를 하나로 통합\n"
            "- 문화/역사, 음식, 실용 정보를 일관되게 정리\n"
            "- 여행자가 바로 사용할 수 있는 완전한 가이드 작성\n"
            "- 각 분야의 핵심 포인트 강조\n"
            "- 일별 추천 일정 제안\n"
            "- 전체적인 여행 팁 및 주의사항 정리\n\n"
            "스타일: 종합적이고 실용적인 여행 가이드, 모든 응답은 한글로 작성"
        )
    )
    
    print(f"✅ Agent 1: {culture_expert.name} (Culture & History Expert) 생성 완료")
    print(f"✅ Agent 2: {food_expert.name} (Food & Dining Expert) 생성 완료")
    print(f"✅ Agent 3: {practical_expert.name} (Practical Travel Expert) 생성 완료")
    print(f"✅ Agent 4: {final_summarizer.name} (Final Summarizer) 생성 완료")
    print("="*70 + "\n")
    
    return culture_expert, food_expert, practical_expert, final_summarizer

# Concurrent Agents 생성
culture_expert, food_expert, practical_expert, final_summarizer = create_concurrent_agents()

# ========================================================================
# Step 1: Executor 노드 정의 (병렬 실행될 전문가 노드들)
# ========================================================================

@executor(id="broadcast_start")
async def broadcast_start_node(msg: TravelAnalysisRequest, ctx: WorkflowContext[TravelAnalysisRequest]) -> None:
    """Start Node: 모든 병렬 노드로 브로드캐스트"""
    print(f"\n📡 Broadcast Start: 모든 전문가에게 작업 배포 중...")
    # 3개의 병렬 노드로 동시에 메시지 전송
    await ctx.send_message(msg, target_id="culture_history_expert")
    await ctx.send_message(msg, target_id="food_dining_expert")
    await ctx.send_message(msg, target_id="practical_tips_expert")
    print(f"✅ Broadcast 완료: 3개 전문가 동시 실행 시작")


@executor(id="culture_history_expert")
async def culture_history_node(msg: TravelAnalysisRequest, ctx: WorkflowContext[TravelAnalysisRequest]) -> None:
    """Concurrent Step 1: Culture & History Expert - 문화/역사 분석"""
    print(f"\n🏛️  병렬 노드 1: Culture & History Expert - 분석 중...")
    
    query = f"""{msg.user_query}에 대한 문화 및 역사 관점의 추천을 제공하세요:

다음을 포함하세요:
- 꼭 방문해야 할 역사적 명소 (박물관, 유적지, 기념관)
- 각 장소의 역사적 의미와 배경
- 문화 체험 활동 (전통 공연, 축제, 워크샵)
- 현지 에티켓 및 문화적 주의사항
- 추천 방문 시간대 및 팁

깊이있는 문화적 경험을 제공하세요."""
    
    # Agent 실행
    thread = culture_expert.get_new_thread()
    result = await culture_expert.run(query, thread=thread)
    response = result.text if hasattr(result, 'text') else str(result)
    
    # Context에 결과 저장하고 aggregator로 전송
    msg.culture_history = response
    await ctx.send_message(msg, target_id="aggregator")
    print(f"✅ 병렬 노드 1 완료: 문화/역사 분석 완료")


@executor(id="food_dining_expert")
async def food_dining_node(msg: TravelAnalysisRequest, ctx: WorkflowContext[TravelAnalysisRequest]) -> None:
    """Concurrent Step 2: Food & Dining Expert - 음식/맛집 분석"""
    print(f"\n🍽️  병렬 노드 2: Food & Dining Expert - 분석 중...")
    
    query = f"""{msg.user_query}에 대한 음식 및 식도락 관점의 추천을 제공하세요:

다음을 포함하세요:
- 꼭 먹어봐야 할 현지 음식 및 요리
- 추천 맛집 (미슐랭, 현지인 맛집, 길거리 음식)
- 각 음식의 특징과 유래
- 식사 예산 가이드 (저가/중가/고가)
- 채식주의자/알러지 대응 정보
- 예약 필요 여부 및 운영 시간

진정한 미식 경험을 제공하세요."""
    
    # Agent 실행
    thread = food_expert.get_new_thread()
    result = await food_expert.run(query, thread=thread)
    response = result.text if hasattr(result, 'text') else str(result)
    
    # Context에 결과 저장하고 aggregator로 전송
    msg.food_dining = response
    await ctx.send_message(msg, target_id="aggregator")
    print(f"✅ 병렬 노드 2 완료: 음식/맛집 분석 완료")


@executor(id="practical_tips_expert")
async def practical_tips_node(msg: TravelAnalysisRequest, ctx: WorkflowContext[TravelAnalysisRequest]) -> None:
    """Concurrent Step 3: Practical Travel Expert - 실용 정보 분석"""
    print(f"\n🧳 병렬 노드 3: Practical Travel Expert - 분석 중...")
    
    query = f"""{msg.user_query}에 대한 실용적인 여행 정보를 제공하세요:

다음을 포함하세요:
- 교통: 공항-시내 이동, 대중교통, 교통카드
- 숙박: 지역별 숙박 추천 (위치, 가격대, 특징)
- 예산: 일일 예산 가이드 (숙박/교통/식사/관광)
- 통신: SIM카드, WiFi, 추천 앱
- 안전: 안전 수칙, 긴급 연락처
- 쇼핑: 추천 쇼핑 지역, 면세점, 기념품
- 날씨: 계절별 옷차림, 우산/선크림 등

실질적이고 유용한 정보를 제공하세요."""
    
    # Agent 실행
    thread = practical_expert.get_new_thread()
    result = await practical_expert.run(query, thread=thread)
    response = result.text if hasattr(result, 'text') else str(result)
    
    # Context에 결과 저장하고 aggregator로 전송
    msg.practical_tips = response
    await ctx.send_message(msg, target_id="aggregator")
    print(f"✅ 병렬 노드 3 완료: 실용 정보 분석 완료")


@executor(id="aggregator")
async def aggregator_node(msg: TravelAnalysisRequest, ctx: WorkflowContext[TravelAnalysisRequest]) -> None:
    """Aggregation Step: 모든 병렬 분석 결과를 통합"""
    print(f"\n📊 Aggregator: 모든 전문가 분석 결과 통합 중...")
    
    # 모든 병렬 분석 결과를 통합하여 최종 요약 생성
    query = f"""여러 전문가의 분석 결과를 통합하여 완벽한 여행 가이드를 작성하세요:

원본 요청: {msg.user_query}

=== 문화/역사 전문가 분석 ===
{msg.culture_history}

=== 음식/맛집 전문가 분석 ===
{msg.food_dining}

=== 실용 정보 전문가 분석 ===
{msg.practical_tips}

위의 모든 정보를 통합하여:
1. 종합적인 여행 가이드 작성
2. 일별 추천 일정 제안 (각 분야의 추천 통합)
3. 각 분야의 핵심 포인트 강조
4. 전체적인 여행 팁 및 주의사항
5. 여행자가 바로 사용할 수 있는 실용적인 형식

완전하고 일관된 최종 여행 가이드를 작성하세요."""
    
    # Final Summarizer Agent 실행
    thread = final_summarizer.get_new_thread()
    result = await final_summarizer.run(query, thread=thread)
    response = result.text if hasattr(result, 'text') else str(result)
    
    # 최종 결과를 Context에 저장하고 출력
    msg.final_summary = response
    await ctx.yield_output(msg)
    print(f"✅ Aggregator 완료: 최종 통합 가이드 작성 완료")


# ========================================================================
# Step 2: WorkflowBuilder로 병렬 실행 그래프 구성
# ========================================================================

concurrent_workflow = (
    WorkflowBuilder()
    # Broadcast 노드에서 시작
    .set_start_executor(broadcast_start_node)
    # Broadcast → 3개 병렬 노드로 fan-out
    .add_edge(broadcast_start_node, culture_history_node)
    .add_edge(broadcast_start_node, food_dining_node)
    .add_edge(broadcast_start_node, practical_tips_node)
    # 3개 병렬 노드 → aggregator로 fan-in
    .add_edge(culture_history_node, aggregator_node)
    .add_edge(food_dining_node, aggregator_node)
    .add_edge(practical_tips_node, aggregator_node)
    .build()
)

print("\n" + "="*70)
print("✅ Concurrent Workflow 그래프 구성 완료")
print("="*70)
print("📊 Workflow 구조 (병렬 패턴):")
print("   broadcast_start_node (시작)")
print("         ↓")
print("   ┌─────┼─────┐")
print("   ↓     ↓     ↓")
print("   culture  food  practical")
print("   history  dining  tips")
print("   (병렬1)  (병렬2)  (병렬3)")
print("   └─────┼─────┘")
print("         ↓")
print("   aggregator_node (통합)")
print("="*70 + "\n")


# ========================================================================
# Step 3: Workflow 실행
# ========================================================================

async def run_concurrent_workflow():
    """Concurrent Workflow 실행 함수"""
    print("\n" + "="*70)
    print("🚀 Concurrent Workflow 실행 시작 (MAF WorkflowBuilder)")
    print("="*70)

    # 입력 메시지 생성
    user_query = "런던 5일 여행 계획을 세워주세요."
    analysis_request = TravelAnalysisRequest(
        destination="런던",
        user_query=user_query
    )

    # Workflow 실행 (run_stream 사용 - async generator)
    outputs = []
    async for event in concurrent_workflow.run_stream(analysis_request):
        # 이벤트에서 output 추출
        if hasattr(event, 'output') and event.output is not None:
            outputs.append(event.output)
            print(f"📤 이벤트 수신: {type(event.output).__name__}")
        elif hasattr(event, 'data') and event.data is not None:
            outputs.append(event.data)
            print(f"📤 이벤트 수신: {type(event.data).__name__}")

    # 최종 결과 추출
    if outputs:
        final_result = outputs[-1]
    else:
        raise ValueError("❌ Workflow에서 출력을 받지 못했습니다.")

    print("\n" + "="*70)
    print("📊 Concurrent Pattern 결과 (MAF WorkflowBuilder)")
    print("="*70)
    print(f"\n📝 여행 요청: {final_result.user_query}\n")
    print(f"\n{'='*70}")
    print(f"🏛️  병렬 분석 1 - Culture & History Expert:")
    print(f"{'='*70}")
    print(f"{final_result.culture_history}\n")
    print(f"\n{'='*70}")
    print(f"🍽️  병렬 분석 2 - Food & Dining Expert:")
    print(f"{'='*70}")
    print(f"{final_result.food_dining}\n")
    print(f"\n{'='*70}")
    print(f"🧳 병렬 분석 3 - Practical Travel Expert:")
    print(f"{'='*70}")
    print(f"{final_result.practical_tips}\n")
    print(f"\n{'='*70}")
    print(f"📋 최종 통합 가이드 - Final Summarizer:")
    print(f"{'='*70}")
    print(f"{final_result.final_summary}")
    print(f"\n{'='*70}")
    
    return final_result

# Workflow 실행
concurrent_result = await run_concurrent_workflow()

**Concurrent Pattern 분석 (MAF WorkflowBuilder 사용)**

### ✅ 핵심 구성 요소

**1. Message Type (데이터 전달)**
- `TravelAnalysisRequest` dataclass로 병렬 분석 결과를 저장할 데이터 구조 정의
- 각 전문가 노드가 독립적으로 자신의 분석 결과를 메시지 객체에 저장

**2. Executor 노드 (@executor 데코레이터)**
- `@executor(id="culture_history_expert")`: 병렬 노드 1 - 문화/역사 분석
- `@executor(id="food_dining_expert")`: 병렬 노드 2 - 음식/맛집 분석
- `@executor(id="practical_tips_expert")`: 병렬 노드 3 - 실용 정보 분석
- `@executor(id="aggregator")`: 통합 노드 - 모든 결과 수집 및 최종 요약

**3. WorkflowBuilder (병렬 실행 그래프 구성)**
```python
workflow = (
    WorkflowBuilder()
    # 3개의 노드를 동시에 시작 (병렬 실행)
    .set_start_executor(culture_history_node)
    .set_start_executor(food_dining_node)
    .set_start_executor(practical_tips_node)
    # 모든 병렬 노드 → aggregator로 수렴
    .add_edge(culture_history_node, aggregator_node)
    .add_edge(food_dining_node, aggregator_node)
    .add_edge(practical_tips_node, aggregator_node)
    .build()
)
```

**4. WorkflowContext (결과 수집 및 통합)**
- 각 병렬 노드: 독립적으로 실행 후 결과를 메시지에 저장
- Aggregator 노드: 모든 병렬 결과를 기다렸다가 통합
- `await ctx.yield_output(msg)`: 최종 통합 결과 출력

### ✅ 실행 흐름

```
TravelAnalysisRequest 생성
    ↓
┌────────────────┬────────────────┬────────────────┐
│ culture_history│  food_dining   │ practical_tips │
│     _node      │     _node      │     _node      │
│    (병렬 1)     │    (병렬 2)     │    (병렬 3)     │
│  Foundry Agent │ Foundry Agent  │ Foundry Agent  │
└────────┬───────┴────────┬───────┴────────┬───────┘
         │                │                │
         └────────────────┼────────────────┘
                          ↓
                  aggregator_node
                  (모든 결과 통합)
                  Foundry Agent
                          ↓
                  최종 통합 가이드
```

### ✅ MAF WorkflowBuilder 병렬 패턴 장점

1. **진정한 병렬 실행**: 여러 `set_start_executor()` 호출로 동시 시작
2. **자동 동기화**: Aggregator 노드가 모든 병렬 노드 완료를 자동으로 대기
3. **타입 안전성**: dataclass로 각 전문가의 결과 필드 명확히 정의
4. **확장성**: 새로운 전문가 노드 추가 용이
5. **성능 향상**: 순차 대비 약 66% 시간 단축 (3개 노드 기준)

### ✅ Sequential Pattern과의 차이점

| 특징 | Sequential Pattern | Concurrent Pattern |
|-----|-------------------|-------------------|
| **노드 시작** | `.set_start_executor()` 1회 | `.set_start_executor()` 다수 호출 |
| **실행 방식** | 순차적 (1→2→3) | 병렬적 (1,2,3 동시) |
| **데이터 흐름** | 이전 노드 결과 → 다음 노드 | 각 노드 독립적 → Aggregator 통합 |
| **사용 사례** | 단계적 개선 필요 | 독립적 분석 필요 |
| **실행 시간** | 합계 (T1+T2+T3) | 최대값 (max(T1,T2,T3)) |

### ✅ 사용 사례
- 다양한 관점에서의 종합 분석 (문화, 음식, 실용 등)
- 독립적인 데이터 수집 작업 (API 병렬 호출)
- 앙상블 의사결정 (여러 모델의 독립적 판단)
- 멀티-에이전트 브레인스토밍
  - 독립적인 정보 수집이 필요한 경우
  - 빠른 응답이 필요한 멀티 도메인 쿼리

**아키텍처 요약**:
```
MAF Concurrent Workflow (병렬 전문가 분석)
         ↓
    asyncio.gather()
    ↙      ↓      ↘
전문가1   전문가2   전문가3
(문화)   (음식)   (실용)
    ↘      ↓      ↙
      결과 통합
```

**실제 사용 예시**:
- 입력: "파리 3일 여행 계획을 세워주세요"
- 전문가 1: 루브르, 에펠탑 등 역사/문화 명소 추천
- 전문가 2: 미슐랭 레스토랑, 비스트로, 카페 추천
- 전문가 3: 메트로 패스, 숙박 지역, 예산 가이드
- 결과: 3가지 관점의 종합적인 여행 정보

## 4. Conditional Branching Pattern (조건부 분기 워크플로우)

In [ ]:
"""
Conditional Branching Pattern - MAF WorkflowBuilder 사용
- @executor 데코레이터로 각 조건부 노드 정의
- WorkflowBuilder로 조건부 분기 그래프 구성
- 여행 스타일에 따라 다른 전문가로 동적 라우팅
"""

# ========================================================================
# Message Type 정의 (Conditional Branching Pattern용)
# ========================================================================

@dataclass
class TravelStyleRequest:
    """여행 스타일별 요청 메시지"""
    user_query: str
    travel_style: Optional[str] = None
    specialized_plan: Optional[str] = None

# ========================================================================
# Conditional Pattern Agents 생성
# ========================================================================

def create_conditional_agents():
    """조건부 분기를 위한 전문가 Agent 생성"""
    
    if not agent_client:
        raise ValueError("❌ MAF Agent client가 초기화되지 않았습니다.")
    
    print("\n" + "="*70)
    print("🤖 조건부 분기 전문가 Agents 생성 중...")
    print("="*70)
    
    # Agent 1: Style Classifier (여행 스타일 분류)
    style_classifier = agent_client.create_agent(
        name="StyleClassifier",
        instructions=(
            "당신은 여행 스타일 분류 전문가입니다.\n\n"
            "역할:\n"
            "- 사용자의 여행 요청을 분석하여 주요 여행 스타일 파악\n"
            "- 다음 중 하나로 정확히 분류:\n"
            "  * 문화: 문화/역사 중심 (박물관, 유적지, 전통)\n"
            "  * 액티비티: 액티비티 중심 (스포츠, 모험, 야외활동)\n"
            "  * 휴양: 휴양 중심 (리조트, 스파, 힐링)\n"
            "  * 미식: 미식 중심 (맛집 투어, 요리 체험, 와이너리)\n\n"
            "중요: 반드시 '문화', '액티비티', '휴양', '미식' 중 하나의 단어로만 응답\n"
            "스타일: 간결하고 정확한 분류, 모든 응답은 한글로 작성"
        )
    )
    
    # Agent 2: Culture Expert (문화/역사 전문가)
    culture_expert = agent_client.create_agent(
        name="CultureExpert",
        instructions=(
            "당신은 문화/역사 여행 전문가입니다.\n\n"
            "역할:\n"
            "- 박물관, 미술관, 역사적 명소 중심 여행 계획\n"
            "- 각 장소의 역사적 의미와 배경 설명\n"
            "- 전통 문화 체험 활동 제안\n"
            "- 최소 5개 이상의 추천 장소 및 활동\n"
            "- 방문 시간, 소요 시간, 예산 가이드 포함\n\n"
            "스타일: 깊이있는 문화 경험 중심, 모든 응답은 한글로 작성"
        )
    )
    
    # Agent 3: Activity Expert (액티비티 전문가)
    activity_expert = agent_client.create_agent(
        name="ActivityExpert",
        instructions=(
            "당신은 액티비티 여행 전문가입니다.\n\n"
            "역할:\n"
            "- 야외 활동, 모험 스포츠, 하이킹 중심 여행 계획\n"
            "- 수상 스포츠, 등산, 사이클링 등 활동적 일정\n"
            "- 각 활동의 난이도와 필요 장비 안내\n"
            "- 최소 5개 이상의 추천 활동 및 장소\n"
            "- 안전 수칙 및 예약 정보 포함\n\n"
            "스타일: 역동적인 경험 중심, 모든 응답은 한글로 작성"
        )
    )
    
    # Agent 4: Relaxation Expert (휴양 전문가)
    relaxation_expert = agent_client.create_agent(
        name="RelaxationExpert",
        instructions=(
            "당신은 휴양 여행 전문가입니다.\n\n"
            "역할:\n"
            "- 리조트, 스파, 힐링 프로그램 중심 여행 계획\n"
            "- 한적한 명소와 자연 경관 추천\n"
            "- 웰니스 프로그램, 명상, 요가 체험 제안\n"
            "- 최소 5개 이상의 추천 휴양지 및 프로그램\n"
            "- 숙박 정보 및 편의시설 안내\n\n"
            "스타일: 편안한 힐링 경험 중심, 모든 응답은 한글로 작성"
        )
    )
    
    # Agent 5: Gourmet Expert (미식 전문가)
    gourmet_expert = agent_client.create_agent(
        name="GourmetExpert",
        instructions=(
            "당신은 미식 여행 전문가입니다.\n\n"
            "역할:\n"
            "- 미슐랭 레스토랑, 맛집 투어 중심 여행 계획\n"
            "- 요리 체험, 와이너리 방문, 푸드 마켓 탐방\n"
            "- 각 레스토랑의 특징과 시그니처 메뉴 소개\n"
            "- 최소 5개 이상의 추천 식당 및 미식 활동\n"
            "- 예약 방법, 드레스 코드, 가격대 안내\n\n"
            "스타일: 진정한 미식 경험 중심, 모든 응답은 한글로 작성"
        )
    )
    
    print(f"✅ Agent 1: {style_classifier.name} (Style Classifier) 생성 완료")
    print(f"✅ Agent 2: {culture_expert.name} (Culture Expert) 생성 완료")
    print(f"✅ Agent 3: {activity_expert.name} (Activity Expert) 생성 완료")
    print(f"✅ Agent 4: {relaxation_expert.name} (Relaxation Expert) 생성 완료")
    print(f"✅ Agent 5: {gourmet_expert.name} (Gourmet Expert) 생성 완료")
    print("="*70 + "\n")
    
    return style_classifier, culture_expert, activity_expert, relaxation_expert, gourmet_expert

# Conditional Agents 생성
style_classifier, culture_expert, activity_expert, relaxation_expert, gourmet_expert = create_conditional_agents()

# ========================================================================
# Step 1: Executor 노드 정의 (조건부 분기)
# ========================================================================

@executor(id="style_classifier")
async def style_classifier_node(msg: TravelStyleRequest, ctx: WorkflowContext[TravelStyleRequest]) -> None:
    """Step 1: 여행 스타일 분류 및 라우팅"""
    print(f"\n🔍 Step 1: Style Classifier - 여행 스타일 분류 중...")
    print(f"   입력: {msg.user_query}")
    
    query = f"""다음 여행 요청을 분석하여 주요 여행 스타일을 한 단어로 분류하세요:

요청: {msg.user_query}

다음 중 하나로 분류하세요:
- 문화: 문화/역사 중심 (박물관, 유적지, 전통)
- 액티비티: 액티비티 중심 (스포츠, 모험, 야외활동)
- 휴양: 휴양 중심 (리조트, 스파, 힐링)
- 미식: 미식 중심 (맛집 투어, 요리 체험, 와이너리)

반드시 '문화', '액티비티', '휴양', '미식' 중 정확히 하나의 단어로만 응답하세요."""
    
    # Style Classifier Agent 실행
    thread = style_classifier.get_new_thread()
    result = await style_classifier.run(query, thread=thread)
    classification = result.text if hasattr(result, 'text') else str(result)
    
    # 분류 결과 정리
    style = classification.strip()
    if "문화" in style or "역사" in style or "박물관" in style:
        style = "문화"
        target_id = "culture_expert"
    elif "액티비티" in style or "모험" in style or "스포츠" in style:
        style = "액티비티"
        target_id = "activity_expert"
    elif "휴양" in style or "힐링" in style or "리조트" in style:
        style = "휴양"
        target_id = "relaxation_expert"
    elif "미식" in style or "맛집" in style or "음식" in style:
        style = "미식"
        target_id = "gourmet_expert"
    else:
        style = "문화"  # 기본값
        target_id = "culture_expert"
    
    # Context에 스타일 저장하고 해당 전문가에게 라우팅
    msg.travel_style = style
    await ctx.send_message(msg, target_id=target_id)
    print(f"✅ 분류 완료: {style} → {target_id}로 라우팅")


@executor(id="culture_expert")
async def culture_expert_node(msg: TravelStyleRequest, ctx: WorkflowContext[TravelStyleRequest]) -> None:
    """Step 2a: 문화/역사 전문가 - 맞춤 계획"""
    print(f"\n🏛️  Step 2a: Culture Expert - 문화/역사 맞춤 계획 수립 중...")
    
    query = f"""문화/역사 여행 스타일에 맞는 상세한 여행 계획을 세워주세요:

요청: {msg.user_query}

중점 사항: 박물관, 미술관, 역사적 명소, 전통 문화 체험

다음을 포함하세요:
- 추천 장소 및 활동 (최소 5개)
- 각 장소의 역사적 의미와 하이라이트
- 방문 시간 및 소요 시간
- 실용적인 팁 및 주의사항
- 예산 가이드

문화/역사 여행 스타일에 완벽히 맞는 계획을 작성하세요."""
    
    # Culture Expert Agent 실행
    thread = culture_expert.get_new_thread()
    result = await culture_expert.run(query, thread=thread)
    response = result.text if hasattr(result, 'text') else str(result)
    
    # 최종 결과 저장 및 출력
    msg.specialized_plan = response
    await ctx.yield_output(msg)
    print(f"✅ Culture Expert 계획 완료")


@executor(id="activity_expert")
async def activity_expert_node(msg: TravelStyleRequest, ctx: WorkflowContext[TravelStyleRequest]) -> None:
    """Step 2b: 액티비티 전문가 - 맞춤 계획"""
    print(f"\n🏃 Step 2b: Activity Expert - 액티비티 맞춤 계획 수립 중...")
    
    query = f"""액티비티 여행 스타일에 맞는 상세한 여행 계획을 세워주세요:

요청: {msg.user_query}

중점 사항: 야외 활동, 모험 스포츠, 하이킹, 수상 스포츠

다음을 포함하세요:
- 추천 활동 및 장소 (최소 5개)
- 각 활동의 난이도 및 필요 장비
- 소요 시간 및 체력 수준
- 안전 수칙 및 주의사항
- 예약 정보 및 예산 가이드

액티비티 여행 스타일에 완벽히 맞는 계획을 작성하세요."""
    
    # Activity Expert Agent 실행
    thread = activity_expert.get_new_thread()
    result = await activity_expert.run(query, thread=thread)
    response = result.text if hasattr(result, 'text') else str(result)
    
    # 최종 결과 저장 및 출력
    msg.specialized_plan = response
    await ctx.yield_output(msg)
    print(f"✅ Activity Expert 계획 완료")


@executor(id="relaxation_expert")
async def relaxation_expert_node(msg: TravelStyleRequest, ctx: WorkflowContext[TravelStyleRequest]) -> None:
    """Step 2c: 휴양 전문가 - 맞춤 계획"""
    print(f"\n🏖️  Step 2c: Relaxation Expert - 휴양 맞춤 계획 수립 중...")
    
    query = f"""휴양 여행 스타일에 맞는 상세한 여행 계획을 세워주세요:

요청: {msg.user_query}

중점 사항: 리조트, 스파, 힐링 프로그램, 한적한 명소

다음을 포함하세요:
- 추천 휴양지 및 프로그램 (최소 5개)
- 각 장소의 특징 및 편의시설
- 웰니스 활동 (스파, 명상, 요가 등)
- 숙박 정보 및 예약 방법
- 예산 가이드

휴양 여행 스타일에 완벽히 맞는 계획을 작성하세요."""
    
    # Relaxation Expert Agent 실행
    thread = relaxation_expert.get_new_thread()
    result = await relaxation_expert.run(query, thread=thread)
    response = result.text if hasattr(result, 'text') else str(result)
    
    # 최종 결과 저장 및 출력
    msg.specialized_plan = response
    await ctx.yield_output(msg)
    print(f"✅ Relaxation Expert 계획 완료")


@executor(id="gourmet_expert")
async def gourmet_expert_node(msg: TravelStyleRequest, ctx: WorkflowContext[TravelStyleRequest]) -> None:
    """Step 2d: 미식 전문가 - 맞춤 계획"""
    print(f"\n🍽️  Step 2d: Gourmet Expert - 미식 맞춤 계획 수립 중...")
    
    query = f"""미식 여행 스타일에 맞는 상세한 여행 계획을 세워주세요:

요청: {msg.user_query}

중점 사항: 미슐랭 레스토랑, 맛집 투어, 요리 체험, 와인 시음

다음을 포함하세요:
- 추천 레스토랑 및 미식 활동 (최소 5개)
- 각 레스토랑의 시그니처 메뉴 및 특징
- 요리 체험, 푸드 마켓 탐방 일정
- 예약 방법, 드레스 코드, 가격대
- 미식 투어 예산 가이드

미식 여행 스타일에 완벽히 맞는 계획을 작성하세요."""
    
    # Gourmet Expert Agent 실행
    thread = gourmet_expert.get_new_thread()
    result = await gourmet_expert.run(query, thread=thread)
    response = result.text if hasattr(result, 'text') else str(result)
    
    # 최종 결과 저장 및 출력
    msg.specialized_plan = response
    await ctx.yield_output(msg)
    print(f"✅ Gourmet Expert 계획 완료")


# ========================================================================
# Step 2: WorkflowBuilder로 조건부 분기 그래프 구성
# ========================================================================

conditional_workflow = (
    WorkflowBuilder()
    # Style Classifier에서 시작
    .set_start_executor(style_classifier_node)
    # Classifier → 4개 전문가 노드로 조건부 라우팅 (실제로는 하나만 실행됨)
    .add_edge(style_classifier_node, culture_expert_node)
    .add_edge(style_classifier_node, activity_expert_node)
    .add_edge(style_classifier_node, relaxation_expert_node)
    .add_edge(style_classifier_node, gourmet_expert_node)
    .build()
)

print("\n" + "="*70)
print("✅ Conditional Workflow 그래프 구성 완료")
print("="*70)
print("📊 Workflow 구조 (조건부 분기 패턴):")
print("   style_classifier_node (분류)")
print("         ↓")
print("   [조건부 라우팅]")
print("   ↙  ↓  ↓  ↘")
print(" 문화 액티 휴양 미식")
print(" 전문가 전문가 전문가 전문가")
print("="*70 + "\n")


# ========================================================================
# Step 3: Workflow 실행
# ========================================================================

async def run_conditional_workflow():
    """Conditional Workflow 실행 함수"""
    print("\n" + "="*70)
    print("🚀 Conditional Workflow 실행 시작 (MAF WorkflowBuilder)")
    print("="*70)

    # 입력 메시지 생성
    user_query = "런던 5일 여행을 계획 중인데, 역사적인 건축물과 박물관을 중심으로 문화를 깊이 체험하고 싶어요. 대영박물관과 타워 브릿지는 꼭 가보고 싶습니다."
    style_request = TravelStyleRequest(user_query=user_query)

    # Workflow 실행 (run_stream 사용 - async generator)
    outputs = []
    async for event in conditional_workflow.run_stream(style_request):
        # 이벤트에서 output 추출
        if hasattr(event, 'output') and event.output is not None:
            outputs.append(event.output)
            print(f"📤 이벤트 수신: {type(event.output).__name__}")
        elif hasattr(event, 'data') and event.data is not None:
            outputs.append(event.data)
            print(f"📤 이벤트 수신: {type(event.data).__name__}")

    # 최종 결과 추출
    if outputs:
        final_result = outputs[-1]
    else:
        raise ValueError("❌ Workflow에서 출력을 받지 못했습니다.")

    print("\n" + "="*70)
    print("📊 Conditional Pattern 결과 (MAF WorkflowBuilder)")
    print("="*70)
    print(f"\n📝 여행 요청: {final_result.user_query}\n")
    print(f"🎯 분류된 여행 스타일: {final_result.travel_style}\n")
    print(f"\n{'='*70}")
    print(f"📋 맞춤형 여행 계획:")
    print(f"{'='*70}")
    print(f"{final_result.specialized_plan}")
    print(f"\n{'='*70}")
    
    return final_result

# Workflow 실행
conditional_result = await run_conditional_workflow()

**Conditional Branching Pattern 분석 (MAF WorkflowBuilder 사용)**

### ✅ 핵심 구성 요소

**1. Message Type (데이터 전달)**
- `TravelStyleRequest` dataclass로 여행 스타일 및 계획 저장
- `travel_style`: 분류된 스타일 ('문화', '액티비티', '휴양', '미식')
- `specialized_plan`: 선택된 전문가의 맞춤 계획

**2. Executor 노드 (@executor 데코레이터)**
- `@executor(id="style_classifier")`: Step 1 - 여행 스타일 분류 및 라우팅
- `@executor(id="culture_expert")`: Step 2a - 문화/역사 전문가 계획
- `@executor(id="activity_expert")`: Step 2b - 액티비티 전문가 계획
- `@executor(id="relaxation_expert")`: Step 2c - 휴양 전문가 계획
- `@executor(id="gourmet_expert")`: Step 2d - 미식 전문가 계획

**3. WorkflowBuilder (조건부 분기 그래프 구성)**
```python
workflow = (
    WorkflowBuilder()
    .set_start_executor(style_classifier_node)  # 분류기에서 시작
    # 분류기 → 4개 전문가 노드로 연결 (조건부 라우팅)
    .add_edge(style_classifier_node, culture_expert_node)
    .add_edge(style_classifier_node, activity_expert_node)
    .add_edge(style_classifier_node, relaxation_expert_node)
    .add_edge(style_classifier_node, gourmet_expert_node)
    .build()
)
```

**4. 동적 라우팅 메커니즘**
- `ctx.send_message(msg, target_id=선택된_전문가)`: 조건에 따라 특정 노드로만 전송
- 분류 결과에 따라 4개 전문가 중 1개만 실행됨

### ✅ 실행 흐름

```
TravelStyleRequest 생성
    ↓
style_classifier_node
(Foundry Agent: style_classifier)
    → 여행 스타일 분석 및 분류
    ↓
[조건부 라우팅]
    ↙  ↓  ↓  ↘
문화   액티비티  휴양   미식
전문가  전문가   전문가  전문가
(하나만 실행됨)
    ↓
specialized_plan 생성
    ↓
최종 결과 출력
```

### ✅ MAF WorkflowBuilder 조건부 패턴 장점

1. **동적 라우팅**: `send_message(target_id=...)`로 런타임에 다음 노드 선택
2. **확장성**: 새로운 스타일/전문가 추가 용이
3. **타입 안전성**: dataclass로 메시지 구조 명확히 정의
4. **효율성**: 필요한 전문가만 실행 (4개 중 1개)
5. **유지보수성**: 각 전문가 노드가 독립적으로 관리됨

### ✅ 조건부 분기 구현 방식

| 방식 | 설명 | 장점 | 단점 |
|-----|------|------|------|
| **send_message(target_id)** | 특정 노드로만 메시지 전송 | 명확한 제어, 효율적 | 수동 라우팅 로직 필요 |
| **add_edge + 조건부 실행** | 모든 엣지 정의, 노드에서 조건 체크 | 그래프 구조 명확 | 모든 노드가 연결됨 |

### ✅ 사용 사례
- 사용자 의도 기반 서비스 라우팅
- 복잡도/우선순위에 따른 티어 분류
- 전문 영역별 전문가 배정
- 맞춤형 추천 시스템

**아키텍처 요약**:
```
사용자 요청
    ↓
분류 에이전트 (스타일 판단)
    ↓
  조건 분기
  ↙ ↓ ↓ ↘
문화 액티비티 휴양 미식
전문가 전문가 전문가 전문가
  ↘ ↓ ↓ ↙
 맞춤 계획 생성
```

**실제 사용 예시**:
- 입력: "유럽의 역사와 예술을 깊이 체험하고 싶어요"
- 분류: "문화" 스타일로 자동 분류
- 라우팅: Culture Expert에게 전달 (다른 3개 전문가는 실행 안 됨)
- 결과: 박물관, 미술관, 유적지 중심의 상세 계획

## 5. Loop-Based Pattern (루프 기반 워크플로우)

In [ ]:
"""
Loop-Based Pattern - MAF WorkflowBuilder 사용
- @executor 데코레이터로 반복 루프 노드 정의
- WorkflowBuilder로 루프 그래프 구성
- 조건부 반복 (최대 3회)으로 일정 개선
"""

# ========================================================================
# Message Type 정의 (Loop Pattern용)
# ========================================================================

@dataclass
class ItineraryRefinementRequest:
    """여행 일정 반복 개선 요청 메시지"""
    destination: str
    days: int
    current_itinerary: Optional[str] = None
    feedback: Optional[str] = None
    feedback_history: Optional[List[str]] = None
    iteration_count: int = 0
    max_iterations: int = 3
    final_itinerary: Optional[str] = None

# ========================================================================
# Loop Pattern Agents 생성
# ========================================================================

def create_loop_agents():
    """반복 개선을 위한 Agent 생성"""
    
    if not agent_client:
        raise ValueError("❌ MAF Agent client가 초기화되지 않았습니다.")
    
    print("\n" + "="*70)
    print("🤖 루프 패턴 Agents 생성 중...")
    print("="*70)
    
    # Agent 1: Itinerary Generator (일정 생성/개선)
    itinerary_generator = agent_client.create_agent(
        name="ItineraryGenerator",
        instructions=(
            "당신은 여행 일정 생성 전문가입니다.\n\n"
            "역할:\n"
            "- 초기 여행 일정 생성 또는 피드백 기반 개선\n"
            "- 일자별 상세 일정 작성\n"
            "- 주요 관광지 및 활동 포함\n"
            "- 식사 추천 및 이동 시간 안내\n"
            "- 예상 비용 제시\n"
            "- 피드백이 있으면 반드시 반영하여 개선\n\n"
            "스타일: 상세하고 실행 가능한 일정, 모든 응답은 한글로 작성"
        )
    )
    
    # Agent 2: Feedback Analyzer (피드백 분석)
    feedback_analyzer = agent_client.create_agent(
        name="FeedbackAnalyzer",
        instructions=(
            "당신은 여행 일정 검토 전문가입니다.\n\n"
            "역할:\n"
            "- 제시된 여행 일정을 다각도로 검토\n"
            "- 시간 배분의 적절성 평가\n"
            "- 동선의 효율성 분석\n"
            "- 활동의 다양성 검토\n"
            "- 휴식 시간 확보 여부 확인\n"
            "- 예산의 현실성 평가\n"
            "- 구체적이고 실행 가능한 개선 제안\n\n"
            "스타일: 건설적이고 구체적인 피드백, 모든 응답은 한글로 작성"
        )
    )
    
    # Agent 3: Final Optimizer (최종 최적화)
    final_optimizer = agent_client.create_agent(
        name="FinalOptimizer",
        instructions=(
            "당신은 여행 일정 최종 검토 전문가입니다.\n\n"
            "역할:\n"
            "- 모든 피드백을 종합하여 최적화된 최종 일정 작성\n"
            "- 개선 과정 요약\n"
            "- 최종 일자별 상세 일정\n"
            "- 주요 하이라이트 강조\n"
            "- 실용적인 팁 제공\n"
            "- 예상 총 비용 정리\n\n"
            "스타일: 완전하고 상세한 최종 계획서, 모든 응답은 한글로 작성"
        )
    )
    
    print(f"✅ Agent 1: {itinerary_generator.name} (Itinerary Generator) 생성 완료")
    print(f"✅ Agent 2: {feedback_analyzer.name} (Feedback Analyzer) 생성 완료")
    print(f"✅ Agent 3: {final_optimizer.name} (Final Optimizer) 생성 완료")
    print("="*70 + "\n")
    
    return itinerary_generator, feedback_analyzer, final_optimizer

# Loop Agents 생성
itinerary_generator, feedback_analyzer, final_optimizer = create_loop_agents()

# ========================================================================
# Step 1: Executor 노드 정의 (루프 패턴)
# ========================================================================

@executor(id="itinerary_generator")
async def itinerary_generator_node(msg: ItineraryRefinementRequest, ctx: WorkflowContext[ItineraryRefinementRequest]) -> None:
    """Step 1: 여행 일정 생성/개선"""
    msg.iteration_count += 1
    is_initial = msg.iteration_count == 1
    
    print(f"\n📋 Step 1: Itinerary Generator - {'초기 일정 생성' if is_initial else f'일정 개선 (라운드 {msg.iteration_count})'}...")
    
    if is_initial:
        query = f"""초기 여행 일정을 생성해주세요.

목적지: {msg.destination}
기간: {msg.days}일

일정은 다음을 포함해야 합니다:
- 일자별 상세 일정
- 주요 관광지 및 활동
- 식사 추천
- 이동 시간 및 방법
- 예상 비용

실행 가능한 상세한 일정을 작성하세요."""
    else:
        query = f"""이전 피드백을 반영하여 개선된 일정을 제시해주세요.

목적지: {msg.destination}
기간: {msg.days}일

이전 일정:
{msg.current_itinerary[:500]}...

최근 피드백:
{msg.feedback[:500]}...

피드백을 반드시 반영하여 일정을 개선하세요."""
    
    # Itinerary Generator Agent 실행
    thread = itinerary_generator.get_new_thread()
    result = await itinerary_generator.run(query, thread=thread)
    response = result.text if hasattr(result, 'text') else str(result)
    
    # 일정 업데이트
    msg.current_itinerary = response
    
    print(f"✅ {'초기 일정' if is_initial else f'개선 일정 (라운드 {msg.iteration_count})'} 생성 완료")
    print(f"\n생성된 일정 미리보기:\n{response[:300]}...\n")
    
    # 최대 반복 횟수 체크
    if msg.iteration_count >= msg.max_iterations:
        # 최종 최적화로 이동
        await ctx.send_message(msg, target_id="final_optimizer")
    else:
        # 피드백 분석으로 이동
        await ctx.send_message(msg, target_id="feedback_analyzer")


@executor(id="feedback_analyzer")
async def feedback_analyzer_node(msg: ItineraryRefinementRequest, ctx: WorkflowContext[ItineraryRefinementRequest]) -> None:
    """Step 2: 피드백 분석 및 개선 제안"""
    print(f"\n💭 Step 2: Feedback Analyzer - 일정 검토 중 (라운드 {msg.iteration_count})...")
    
    query = f"""다음 일정을 검토하고 개선점을 제시해주세요:

목적지: {msg.destination}, {msg.days}일

현재 일정:
{msg.current_itinerary[:600]}...

다음 관점에서 검토하세요:
1. 시간 배분의 적절성
2. 동선의 효율성
3. 활동의 다양성
4. 휴식 시간 확보
5. 예산의 현실성

구체적이고 실행 가능한 개선 제안을 한글로 제시하세요."""
    
    # Feedback Analyzer Agent 실행
    thread = feedback_analyzer.get_new_thread()
    result = await feedback_analyzer.run(query, thread=thread)
    response = result.text if hasattr(result, 'text') else str(result)
    
    # 피드백 저장
    msg.feedback = response
    if msg.feedback_history is None:
        msg.feedback_history = []
    msg.feedback_history.append(response)
    
    print(f"✅ 피드백 분석 완료 (라운드 {msg.iteration_count})")
    print(f"\n피드백 미리보기:\n{response[:300]}...\n")
    
    # 다시 일정 생성으로 루프백
    await ctx.send_message(msg, target_id="itinerary_generator")


@executor(id="final_optimizer")
async def final_optimizer_node(msg: ItineraryRefinementRequest, ctx: WorkflowContext[ItineraryRefinementRequest]) -> None:
    """Step 3: 최종 최적화 및 완성"""
    print(f"\n🎯 Step 3: Final Optimizer - 최종 최적화 중...")
    
    feedback_summary = "\n".join([f"- 라운드 {i+1}: {fb[:200]}..." 
                                   for i, fb in enumerate(msg.feedback_history or [])])
    
    query = f"""지금까지의 피드백을 모두 반영한 최종 최적화 일정을 제시해주세요.

목적지: {msg.destination}
기간: {msg.days}일
개선 반복 횟수: {msg.iteration_count}회

마지막 일정:
{msg.current_itinerary[:600]}...

개선 피드백 히스토리:
{feedback_summary}

최종 일정에는 다음이 포함되어야 합니다:
✅ 개선 과정 요약
✅ 최종 일자별 상세 일정
✅ 주요 하이라이트
✅ 실용적인 팁
✅ 예상 총 비용

반드시 한글로 완전하고 상세하게 답변하세요."""
    
    # Final Optimizer Agent 실행
    thread = final_optimizer.get_new_thread()
    result = await final_optimizer.run(query, thread=thread)
    response = result.text if hasattr(result, 'text') else str(result)
    
    # 최종 결과 저장 및 출력
    msg.final_itinerary = response
    await ctx.yield_output(msg)
    print(f"✅ 최종 최적화 완료")


# ========================================================================
# Step 2: WorkflowBuilder로 루프 그래프 구성
# ========================================================================

loop_workflow = (
    WorkflowBuilder()
    # Itinerary Generator에서 시작
    .set_start_executor(itinerary_generator_node)
    # Generator → Feedback Analyzer (조건부: iteration < max)
    .add_edge(itinerary_generator_node, feedback_analyzer_node)
    # Feedback Analyzer → Generator (루프백)
    .add_edge(feedback_analyzer_node, itinerary_generator_node)
    # Generator → Final Optimizer (조건부: iteration >= max)
    .add_edge(itinerary_generator_node, final_optimizer_node)
    .build()
)

print("\n" + "="*70)
print("✅ Loop-Based Workflow 그래프 구성 완료")
print("="*70)
print("📊 Workflow 구조 (루프 패턴):")
print("   itinerary_generator_node (시작)")
print("         ↓")
print("   [iteration < max?]")
print("         ↓ Yes")
print("   feedback_analyzer_node")
print("         ↓")
print("   [루프백]")
print("         ↓")
print("   itinerary_generator_node")
print("         ↓ (반복)")
print("   [iteration >= max?]")
print("         ↓ Yes")
print("   final_optimizer_node (완료)")
print("="*70 + "\n")


# ========================================================================
# Step 3: Workflow 실행
# ========================================================================

async def run_loop_workflow():
    """Loop Workflow 실행 함수"""
    print("\n" + "="*70)
    print("🚀 Loop-Based Workflow 실행 시작 (MAF WorkflowBuilder)")
    print("="*70)
    print(f"🔄 최대 반복 횟수: 3회")
    print(f"📌 패턴: 일정 생성 → 피드백 → 개선 → (반복) → 최종 최적화")
    print("="*70)

    # 입력 메시지 생성
    refinement_request = ItineraryRefinementRequest(
        destination="런던",
        days=5,
        max_iterations=3
    )

    # Workflow 실행 (run_stream 사용 - async generator)
    outputs = []
    async for event in loop_workflow.run_stream(refinement_request):
        # 이벤트에서 output 추출
        if hasattr(event, 'output') and event.output is not None:
            outputs.append(event.output)
            print(f"📤 이벤트 수신: {type(event.output).__name__}")
        elif hasattr(event, 'data') and event.data is not None:
            outputs.append(event.data)
            print(f"📤 이벤트 수신: {type(event.data).__name__}")

    # 최종 결과 추출
    if outputs:
        final_result = outputs[-1]
    else:
        raise ValueError("❌ Workflow에서 출력을 받지 못했습니다.")

    print("\n" + "="*70)
    print("📊 Loop-Based Pattern 결과 (MAF WorkflowBuilder)")
    print("="*70)
    print(f"\n📝 목적지: {final_result.destination}")
    print(f"📅 기간: {final_result.days}일")
    print(f"🔄 개선 반복 횟수: {final_result.iteration_count}회")
    print(f"💭 피드백 히스토리: {len(final_result.feedback_history or [])}건")
    
    print(f"\n{'='*70}")
    print(f"📋 최종 최적화 일정:")
    print(f"{'='*70}")
    print(f"{final_result.final_itinerary}")
    print(f"\n{'='*70}")
    
    if final_result.feedback_history:
        print(f"\n{'='*70}")
        print(f"💭 개선 피드백 히스토리:")
        print(f"{'='*70}")
        for i, feedback in enumerate(final_result.feedback_history, 1):
            print(f"\n[라운드 {i} 피드백]")
            print(feedback[:400] + "..." if len(feedback) > 400 else feedback)
        print(f"\n{'='*70}")
    
    return final_result

# Workflow 실행
loop_result = await run_loop_workflow()

**Loop-Based Pattern 분석 (MAF WorkflowBuilder 사용)**

### ✅ 핵심 구성 요소

**1. Message Type (데이터 전달 및 상태 관리)**
- `ItineraryRefinementRequest` dataclass로 루프 상태 추적
- `current_itinerary`: 현재 일정 (각 반복마다 업데이트)
- `feedback`: 최근 피드백
- `feedback_history`: 모든 피드백 히스토리
- `iteration_count`: 현재 반복 횟수
- `max_iterations`: 최대 반복 횟수 (3회)

**2. Executor 노드 (@executor 데코레이터)**
- `@executor(id="itinerary_generator")`: 일정 생성/개선 (루프 가능)
- `@executor(id="feedback_analyzer")`: 피드백 분석 (루프 가능)
- `@executor(id="final_optimizer")`: 최종 최적화 (루프 종료)

**3. WorkflowBuilder (루프 그래프 구성)**
```python
workflow = (
    WorkflowBuilder()
    .set_start_executor(itinerary_generator_node)
    # 루프 구조: Generator ↔ Analyzer
    .add_edge(itinerary_generator_node, feedback_analyzer_node)
    .add_edge(feedback_analyzer_node, itinerary_generator_node)
    # 종료 조건: Generator → Optimizer
    .add_edge(itinerary_generator_node, final_optimizer_node)
    .build()
)
```

**4. 루프 제어 메커니즘**
- `iteration_count` 증가 추적
- 조건부 라우팅:
  - `iteration < max` → Feedback Analyzer (계속 루프)
  - `iteration >= max` → Final Optimizer (루프 종료)

### ✅ 실행 흐름

```
ItineraryRefinementRequest 생성 (iteration=0)
    ↓
itinerary_generator_node (iteration=1)
    → 초기 일정 생성
    ↓
[iteration < max?] Yes
    ↓
feedback_analyzer_node
    → 피드백 생성
    ↓
[루프백] itinerary_generator_node (iteration=2)
    → 피드백 반영한 개선 일정
    ↓
[iteration < max?] Yes
    ↓
feedback_analyzer_node
    → 피드백 생성
    ↓
[루프백] itinerary_generator_node (iteration=3)
    → 피드백 반영한 개선 일정
    ↓
[iteration >= max?] Yes
    ↓
final_optimizer_node
    → 최종 최적화 일정
    ↓
최종 결과 출력
```

### ✅ MAF WorkflowBuilder 루프 패턴 장점

1. **명확한 루프 구조**: 엣지로 루프 경로 명시
2. **상태 추적**: dataclass의 `iteration_count`로 진행 상황 관리
3. **조건부 종료**: 노드 내에서 조건 체크 후 다른 경로로 라우팅
4. **히스토리 관리**: `feedback_history`로 모든 개선 과정 기록
5. **유연성**: `max_iterations` 값 변경으로 반복 횟수 조절

### ✅ 루프 구현 패턴

| 구성 요소 | 역할 | 구현 방법 |
|---------|------|---------|
| **카운터** | 반복 횟수 추적 | `msg.iteration_count += 1` |
| **종료 조건** | 루프 탈출 결정 | `if iteration >= max` |
| **루프백** | 다시 시작 노드로 | `send_message(target_id="generator")` |
| **상태 전달** | 이전 결과 유지 | dataclass 필드 업데이트 |

### ✅ 사용 사례
- 반복 개선이 필요한 문서 작성
- 코드 리뷰 및 수정 사이클
- 디자인 피드백 반영
- 품질 검증 및 개선
- 학습 및 최적화 과정

**아키텍처 요약**:
```
초기 요청
    ↓
┌─→ 일정 생성 ─→ [종료?] ─Yes→ 최종 최적화
│       ↓           ↑
│      No           │
│       ↓           │
└─ 피드백 분석 ─────┘
   (루프백)
```

**실제 사용 예시**:
- 입력: "런던 5일 여행 일정"
- 라운드 1: 초기 일정 생성 → 피드백 (동선 개선 필요)
- 라운드 2: 동선 개선 일정 → 피드백 (휴식 시간 추가 필요)
- 라운드 3: 휴식 추가 일정 → 최종 최적화
- 결과: 3회 개선을 거친 최적화된 최종 일정

---

## 6. Error Handling & Retry Pattern (오류 처리 및 재시도)

In [ ]:
"""
Error Handling Pattern - MAF WorkflowBuilder 사용
- @executor 데코레이터로 에러 처리 노드 정의
- WorkflowBuilder로 에러 복구 그래프 구성
- 실패 감지 → 분석 → 대안 제시 → 복구
"""

# ========================================================================
# Message Type 정의 (Error Handling Pattern용)
# ========================================================================

@dataclass
class BookingRequest:
    """예약 요청 메시지 (에러 핸들링 포함)"""
    destination: str
    dates: str
    hotel_name: str
    simulate_error: bool = False
    booking_result: Optional[str] = None
    error_detected: bool = False
    error_analysis: Optional[str] = None
    fallback_options: Optional[str] = None
    final_booking: Optional[str] = None
    status: str = "pending"

# ========================================================================
# Error Handling Pattern Agents 생성
# ========================================================================

def create_error_handling_agents():
    """에러 핸들링을 위한 Agent 생성"""
    
    if not agent_client:
        raise ValueError("❌ MAF Agent client가 초기화되지 않았습니다.")
    
    print("\n" + "="*70)
    print("🤖 에러 핸들링 패턴 Agents 생성 중...")
    print("="*70)
    
    # Agent 1: Booking Agent (예약 시도)
    booking_agent = agent_client.create_agent(
        name="BookingAgent",
        instructions=(
            "당신은 여행 예약 전문가입니다.\n\n"
            "역할:\n"
            "- 호텔 및 항공권 예약 처리\n"
            "- 예약 가능 여부 확인\n"
            "- 호텔 정보 및 가격 제공\n"
            "- 예약 결과 상세 안내\n"
            "- 특이사항 및 주의사항 전달\n\n"
            "스타일: 정확하고 상세한 예약 정보, 모든 응답은 한글로 작성"
        )
    )
    
    # Agent 2: Error Analyzer (오류 분석)
    error_analyzer = agent_client.create_agent(
        name="ErrorAnalyzer",
        instructions=(
            "당신은 예약 시스템 오류 분석 전문가입니다.\n\n"
            "역할:\n"
            "- 예약 실패 원인 심층 분석\n"
            "- 실패 유형 분류 (만실, 가격 변동, 시스템 오류 등)\n"
            "- 대안 가능성 평가\n"
            "- 해결 방안 추천\n"
            "- 고객 영향도 평가\n\n"
            "스타일: 체계적이고 건설적인 분석, 모든 응답은 한글로 작성"
        )
    )
    
    # Agent 3: Fallback Planner (대안 제시)
    fallback_planner = agent_client.create_agent(
        name="FallbackPlanner",
        instructions=(
            "당신은 여행 대안 기획 전문가입니다.\n\n"
            "역할:\n"
            "- 실패한 예약에 대한 실용적인 대안 제시\n"
            "- 같은 지역 다른 호텔 추천 (3개 이상)\n"
            "- 날짜 변경 옵션 제안\n"
            "- 인근 대체 도시 옵션 제공\n"
            "- 각 대안의 장단점 비교\n"
            "- 가격 및 품질 정보 포함\n\n"
            "스타일: 구체적이고 실행 가능한 대안, 모든 응답은 한글로 작성"
        )
    )
    
    # Agent 4: Recovery Agent (복구 및 재예약)
    recovery_agent = agent_client.create_agent(
        name="RecoveryAgent",
        instructions=(
            "당신은 예약 복구 전문가입니다.\n\n"
            "역할:\n"
            "- 최적의 대안 선택 및 추천\n"
            "- 재예약 진행 및 확정\n"
            "- 최종 예약 정보 정리\n"
            "- 고객 안내 사항 작성\n"
            "- 원래 계획과의 차이점 설명\n\n"
            "스타일: 안심시키고 명확한 최종 안내, 모든 응답은 한글로 작성"
        )
    )
    
    print(f"✅ Agent 1: {booking_agent.name} (Booking Agent) 생성 완료")
    print(f"✅ Agent 2: {error_analyzer.name} (Error Analyzer) 생성 완료")
    print(f"✅ Agent 3: {fallback_planner.name} (Fallback Planner) 생성 완료")
    print(f"✅ Agent 4: {recovery_agent.name} (Recovery Agent) 생성 완료")
    print("="*70 + "\n")
    
    return booking_agent, error_analyzer, fallback_planner, recovery_agent

# Error Handling Agents 생성
booking_agent, error_analyzer, fallback_planner, recovery_agent = create_error_handling_agents()

# ========================================================================
# Step 1: Executor 노드 정의 (에러 핸들링 패턴)
# ========================================================================

@executor(id="booking_agent")
async def booking_agent_node(msg: BookingRequest, ctx: WorkflowContext[BookingRequest]) -> None:
    """Step 1: 예약 시도"""
    print(f"\n🔄 Step 1: Booking Agent - 예약 시도 중...")
    
    if msg.simulate_error:
        query = f"""[오류 시뮬레이션] 다음 예약은 실패했습니다:

목적지: {msg.destination}
날짜: {msg.dates}
호텔: {msg.hotel_name}

예약이 불가능한 이유를 설명하세요:
- 만실 상태
- 가격 급등
- 또는 기타 사유

반드시 한글로 답변하세요."""
    else:
        query = f"""다음 예약을 진행해주세요:

목적지: {msg.destination}
날짜: {msg.dates}
호텔: {msg.hotel_name}

예약 결과를 다음 형식으로 제시하세요:
- 예약 가능 여부: 가능/불가능
- 호텔 정보: 등급, 위치, 특징
- 가격 정보: 1박 가격, 총 가격
- 특이사항: 조식 포함, 취소 정책 등

반드시 한글로 답변하세요."""
    
    # Booking Agent 실행
    thread = booking_agent.get_new_thread()
    result = await booking_agent.run(query, thread=thread)
    response = result.text if hasattr(result, 'text') else str(result)
    
    # 예약 결과 저장
    msg.booking_result = response
    
    print(f"📋 예약 결과 미리보기:\n{response[:300]}...\n")
    
    # 에러 감지 (키워드 기반)
    error_keywords = ["불가능", "실패", "만실", "없습니다", "불가", "못"]
    msg.error_detected = any(keyword in response for keyword in error_keywords)
    
    if msg.error_detected:
        print(f"⚠️  에러 감지! Error Analyzer로 전달")
        msg.status = "error_detected"
        await ctx.send_message(msg, target_id="error_analyzer")
    else:
        print(f"✅ 예약 성공!")
        msg.status = "success"
        await ctx.yield_output(msg)


@executor(id="error_analyzer")
async def error_analyzer_node(msg: BookingRequest, ctx: WorkflowContext[BookingRequest]) -> None:
    """Step 2: 에러 분석"""
    print(f"\n🔍 Step 2: Error Analyzer - 오류 원인 분석 중...")
    
    query = f"""예약 실패 원인을 심층 분석하고 대안 방향을 제시해주세요.

실패한 예약 정보:
- 목적지: {msg.destination}
- 날짜: {msg.dates}
- 호텔: {msg.hotel_name}

실패 상황:
{msg.booking_result[:500]}...

다음을 분석하세요:
1. 주요 실패 원인 (만실, 가격 상승, 시스템 오류 등)
2. 실패 유형 분류
3. 대안 가능성 평가
4. 추천 해결 방안

구체적이고 실용적인 분석을 한글로 제시하세요."""
    
    # Error Analyzer Agent 실행
    thread = error_analyzer.get_new_thread()
    result = await error_analyzer.run(query, thread=thread)
    response = result.text if hasattr(result, 'text') else str(result)
    
    # 분석 결과 저장
    msg.error_analysis = response
    msg.status = "error_analyzed"
    
    print(f"✅ 오류 분석 완료")
    print(f"분석 결과 미리보기:\n{response[:300]}...\n")
    
    # Fallback Planner로 전달
    await ctx.send_message(msg, target_id="fallback_planner")


@executor(id="fallback_planner")
async def fallback_planner_node(msg: BookingRequest, ctx: WorkflowContext[BookingRequest]) -> None:
    """Step 3: 대안 옵션 제시"""
    print(f"\n🎯 Step 3: Fallback Planner - 대안 옵션 제시 중...")
    
    query = f"""실패한 예약에 대한 구체적인 대안을 3가지 이상 제시해주세요.

원래 계획:
- 목적지: {msg.destination}
- 날짜: {msg.dates}
- 호텔: {msg.hotel_name}

실패 원인 분석:
{msg.error_analysis[:400]}...

다음 대안들을 제시하세요:
1. 같은 지역 다른 호텔 (최소 3개, 다양한 가격대)
2. 날짜 변경 옵션 (가능하면)
3. 인근 대체 도시 옵션
4. 각 대안의 장단점 및 가격 비교

실용적이고 구체적인 대안을 한글로 제시하세요."""
    
    # Fallback Planner Agent 실행
    thread = fallback_planner.get_new_thread()
    result = await fallback_planner.run(query, thread=thread)
    response = result.text if hasattr(result, 'text') else str(result)
    
    # 대안 저장
    msg.fallback_options = response
    msg.status = "fallback_planned"
    
    print(f"✅ 대안 제시 완료")
    print(f"대안 미리보기:\n{response[:300]}...\n")
    
    # Recovery Agent로 전달
    await ctx.send_message(msg, target_id="recovery_agent")


@executor(id="recovery_agent")
async def recovery_agent_node(msg: BookingRequest, ctx: WorkflowContext[BookingRequest]) -> None:
    """Step 4: 최적 대안 선택 및 복구"""
    print(f"\n✅ Step 4: Recovery Agent - 최적 대안 선택 및 재예약 중...")
    
    query = f"""최적의 대안을 선택하고 재예약을 완료해주세요.

원래 계획:
- 목적지: {msg.destination}
- 날짜: {msg.dates}
- 호텔: {msg.hotel_name}

제시된 대안:
{msg.fallback_options[:600]}...

다음을 수행하세요:
1. 최적 대안 선택 및 선택 이유 설명
2. 재예약 진행 상황 (가상으로 완료)
3. 최종 예약 확정 정보 (호텔명, 가격, 특징)
4. 원래 계획과의 차이점 안내
5. 고객 안내 사항 (체크인, 교통편 등)

안심시키고 명확한 최종 안내를 한글로 작성하세요."""
    
    # Recovery Agent 실행
    thread = recovery_agent.get_new_thread()
    result = await recovery_agent.run(query, thread=thread)
    response = result.text if hasattr(result, 'text') else str(result)
    
    # 최종 복구 결과 저장 및 출력
    msg.final_booking = response
    msg.status = "recovered"
    
    print(f"✅ 복구 완료")
    
    await ctx.yield_output(msg)


# ========================================================================
# Step 2: WorkflowBuilder로 에러 핸들링 그래프 구성
# ========================================================================

error_handling_workflow = (
    WorkflowBuilder()
    # Booking Agent에서 시작
    .set_start_executor(booking_agent_node)
    # Booking Agent → Error Analyzer (에러 발생 시)
    .add_edge(booking_agent_node, error_analyzer_node)
    # Error Analyzer → Fallback Planner
    .add_edge(error_analyzer_node, fallback_planner_node)
    # Fallback Planner → Recovery Agent
    .add_edge(fallback_planner_node, recovery_agent_node)
    .build()
)

print("\n" + "="*70)
print("✅ Error Handling Workflow 그래프 구성 완료")
print("="*70)
print("📊 Workflow 구조 (에러 핸들링 패턴):")
print("   booking_agent_node (예약 시도)")
print("         ↓")
print("   [에러 발생?]")
print("         ↓ Yes")
print("   error_analyzer_node (오류 분석)")
print("         ↓")
print("   fallback_planner_node (대안 제시)")
print("         ↓")
print("   recovery_agent_node (복구)")
print("="*70 + "\n")


# ========================================================================
# Step 3: Workflow 실행
# ========================================================================

async def run_error_handling_workflow(simulate_error: bool = True):
    """Error Handling Workflow 실행 함수"""
    print("\n" + "="*70)
    print("🚀 Error Handling Workflow 실행 시작 (MAF WorkflowBuilder)")
    print("="*70)
    print(f"🧪 오류 시뮬레이션: {'활성화' if simulate_error else '비활성화'}")
    print("="*70)

    # 입력 메시지 생성
    booking_request = BookingRequest(
        destination="런던",
        dates="2025-03-15 ~ 2025-03-19",
        hotel_name="힐튼 런던 패딩턴",
        simulate_error=simulate_error
    )

    # Workflow 실행 (run_stream 사용 - async generator)
    outputs = []
    async for event in error_handling_workflow.run_stream(booking_request):
        # 이벤트에서 output 추출
        if hasattr(event, 'output') and event.output is not None:
            outputs.append(event.output)
            print(f"📤 이벤트 수신: {type(event.output).__name__}")
        elif hasattr(event, 'data') and event.data is not None:
            outputs.append(event.data)
            print(f"📤 이벤트 수신: {type(event.data).__name__}")

    # 최종 결과 추출
    if outputs:
        final_result = outputs[-1]
    else:
        raise ValueError("❌ Workflow에서 출력을 받지 못했습니다.")

    print("\n" + "="*70)
    print("📊 Error Handling Pattern 결과 (MAF WorkflowBuilder)")
    print("="*70)
    print(f"\n📝 예약 정보:")
    print(f"   - 목적지: {final_result.destination}")
    print(f"   - 날짜: {final_result.dates}")
    print(f"   - 호텔: {final_result.hotel_name}")
    print(f"\n✅ 최종 상태: {final_result.status}")
    print(f"⚠️  오류 감지: {'예' if final_result.error_detected else '아니오'}")
    
    if final_result.error_detected:
        print(f"\n{'='*70}")
        print(f"🔍 오류 분석:")
        print(f"{'='*70}")
        print(f"{final_result.error_analysis}\n")
        
        print(f"\n{'='*70}")
        print(f"🎯 대안 옵션:")
        print(f"{'='*70}")
        print(f"{final_result.fallback_options}\n")
        
        print(f"\n{'='*70}")
        print(f"✅ 최종 복구 결과:")
        print(f"{'='*70}")
        print(f"{final_result.final_booking}")
    else:
        print(f"\n{'='*70}")
        print(f"✅ 예약 결과:")
        print(f"{'='*70}")
        print(f"{final_result.booking_result}")
    
    print(f"\n{'='*70}")
    
    return final_result

# Workflow 실행 (오류 시뮬레이션 활성화)
print("\n🧪 시나리오: 예약 실패 상황 (오류 처리 테스트)")
error_result = await run_error_handling_workflow(simulate_error=True)

### Error Handling Pattern 분석 및 정리

#### 📊 에러 처리 패턴의 장점

1. **자동 오류 감지 및 복구**
   - 키워드 기반으로 실패 자동 감지
   - 에이전트가 원인 분석 및 대안 제시
   - 사용자 개입 없이 자동 복구 시도
   - 완전한 예약 실패 방지

2. **단계별 오류 처리**
   - **Booking Node**: 초기 시도 및 실패 감지
   - **Error Analyzer**: 실패 원인 상세 분석
   - **Fallback Planner**: 대안 옵션 생성
   - **Recovery Node**: 최종 복구 및 예약 완료

3. **사용자 경험 개선**
   - 투명한 오류 메시지
   - 명확한 대안 제시
   - 자동 복구로 시간 절약
   - 고객 만족도 향상

4. **시스템 신뢰성 향상**
   - 예외 상황에서도 안정적 동작
   - 비즈니스 연속성 보장
   - 에러 패턴 추적 및 개선
   - 프로덕션 환경에 적합

#### 🎯 실제 적용 사례

**여행 예약 시스템**
```
정상 흐름: 예약 시도 → 성공 → 확인
실패 흐름: 예약 시도 → 실패 → 분석 → 대안 → 복구
```

**전자상거래**
```
정상: 주문 → 결제 → 배송
실패: 주문 → 재고 부족 → 대체 상품 추천 → 완료
```

**API 통합 시스템**
```
정상: API 호출 → 성공
실패: API 호출 → 타임아웃 → 재시도 → 캐시 데이터 → 성공
```

**금융 거래**
```
정상: 거래 요청 → 승인 → 완료
실패: 거래 요청 → 한도 초과 → 분할 거래 제안 → 승인 → 완료
```

#### 💡 에러 처리 패턴 구현 팁

**1. 명확한 실패 기준 정의**
```python
# 키워드 기반 감지
error_keywords = ["실패", "불가능", "예약할 수 없", "만석", "sold out"]

# 상태 코드 기반
if status_code in [400, 404, 500]:
    trigger_error_handling()
```

**2. 상세한 오류 분석**
```python
# 실패 원인 분류
- 재고/가용성 문제
- 시스템 오류
- 입력 데이터 문제
- 외부 의존성 실패
```

**3. 다단계 폴백 전략**
```python
Level 1: 자동 재시도 (3회)
Level 2: 대안 옵션 제시
Level 3: 사용자 확인 요청
Level 4: 수동 처리로 에스컬레이션
```

**4. 로깅 및 모니터링**
```python
logger.warning(f"예약 실패 감지: {error_type}")
logger.info(f"대안 옵션 생성: {alternatives}")
logger.info(f"복구 성공: {recovery_result}")

# 메트릭 수집
metrics.increment('booking.failures')
metrics.increment('booking.recoveries')
```

#### 🚨 주의사항

⚠️ **무한 재시도 방지**
- 최대 재시도 횟수 설정 (예: 3회)
- 지수 백오프(exponential backoff) 사용
- 최종적으로 실패 시 명확한 에러 메시지

⚠️ **순환 참조 방지**
- Error Analyzer → Fallback → Recovery (단방향)
- 복구 실패 시 무한 루프 방지

⚠️ **비용 관리**
- 재시도마다 API 호출 비용 발생
- 타임아웃 설정으로 비용 제한
- 폴백 옵션은 간단하게 유지

#### 📈 성능 지표

| 지표 | 목표 | 설명 |
|------|------|------|
| **Recovery Rate** | > 80% | 자동 복구 성공률 |
| **Time to Recovery** | < 30초 | 평균 복구 소요 시간 |
| **False Positive** | < 5% | 정상을 실패로 잘못 감지 |
| **User Satisfaction** | > 4.5/5 | 복구 후 고객 만족도 |

---

## 7. Handoff Pattern (핸드오프 패턴) - 동적 에이전트 전환

### 핸드오프 패턴이란?

핸드오프(Handoff) 패턴은 작업의 복잡도나 특성에 따라 **다른 전문 에이전트로 동적으로 제어를 이전**하는 패턴입니다.

```
Initial Agent → [평가] → 조건 1? → Specialist A
                      → 조건 2? → Specialist B  
                      → 조건 3? → Specialist C
                      → 기본 → Continue
```

### 핸드오프 패턴의 핵심 개념

1. **트리거 조건 (Trigger Conditions)**
   - 복잡도 기준: 간단한 요청 vs 복잡한 요청
   - 전문성 기준: 일반 지식 vs 전문 지식
   - 서비스 레벨: 기본 서비스 vs 프리미엄 서비스

2. **컨텍스트 전달 (Context Transfer)**
   - 이전 대화 내역 전달
   - 중요 정보 추출 및 전달
   - 핸드오프 이유 명시

3. **역할 분리 (Role Separation)**
   - 초기 분류 에이전트 (Triage Agent)
   - 전문가 에이전트들 (Specialist Agents)
   - 최종 통합 에이전트 (Coordinator Agent)

### 사용 사례

- **여행 계획**: 간단한 여행 → 맞춤형 여행 → 럭셔리 여행 → VIP 컨시어지
- **고객 지원**: 일반 문의 → 복잡한 문의 → 기술 전문가 → 매니저
- **의료 시스템**: 증상 체크 → 일반의 → 전문의 → 수술팀
- **금융 서비스**: 간단한 조회 → 복잡한 거래 → 승인 필요 → 매니저 승인

In [ ]:
"""
Handoff Pattern - MAF WorkflowBuilder 사용
- @executor 데코레이터로 핸드오프 노드 정의
- WorkflowBuilder로 동적 라우팅 그래프 구성
- Triage → 복잡도 평가 → 전문가 선택 → Coordinator
"""

# ========================================================================
# Message Type 정의 (Handoff Pattern용)
# ========================================================================

@dataclass
class TravelPlanRequest:
    """여행 계획 요청 메시지 (핸드오프 포함)"""
    travel_request: str
    complexity_score: int = 0
    triage_analysis: Optional[str] = None
    assigned_specialist: str = "미정"
    specialist_response: Optional[str] = None
    final_proposal: Optional[str] = None
    status: str = "pending"

# ========================================================================
# Handoff Pattern Agents 생성
# ========================================================================

def create_handoff_agents():
    """핸드오프를 위한 Agent 생성"""
    
    if not agent_client:
        raise ValueError("❌ MAF Agent client가 초기화되지 않았습니다.")
    
    print("\n" + "="*70)
    print("🤖 핸드오프 패턴 Agents 생성 중...")
    print("="*70)
    
    # Agent 1: Triage Agent (분류 및 복잡도 평가)
    triage_agent = agent_client.create_agent(
        name="TriageAgent",
        instructions=(
            "당신은 여행 요청 분류 전문가입니다.\n\n"
            "역할:\n"
            "- 여행 요청의 복잡도 정확히 평가 (0-100점)\n"
            "- 필요한 전문성 수준 판단\n"
            "- 적절한 전문가 타입 추천\n"
            "- 키워드 및 특성 분석\n\n"
            "평가 기준:\n"
            "- 0-30: 간단한 여행 (주요 도시, 짧은 기간, 일반 관광)\n"
            "- 31-70: 맞춤형 여행 (테마, 여러 도시, 특정 관심사)\n"
            "- 71-100: 럭셔리/복잡한 여행 (프라이빗 투어, 고급 숙소, 특별 경험)\n\n"
            "스타일: 정확하고 객관적인 분석, 모든 응답은 한글로 작성"
        )
    )
    
    # Agent 2: General Travel Advisor (일반 여행 상담)
    general_advisor = agent_client.create_agent(
        name="GeneralAdvisor",
        instructions=(
            "당신은 친절한 일반 여행 상담사입니다.\n\n"
            "역할:\n"
            "- 간단하고 실용적인 여행 계획 제공\n"
            "- 주요 관광지 3-5곳 추천\n"
            "- 중급 호텔 및 대중교통 정보\n"
            "- 대략적인 예산 가이드\n"
            "- 인기 식당 및 카페 추천\n\n"
            "스타일: 쉽고 친근한 설명, 모든 응답은 한글로 작성"
        )
    )
    
    # Agent 3: Specialist Travel Planner (전문 여행 기획자)
    specialist_planner = agent_client.create_agent(
        name="SpecialistPlanner",
        instructions=(
            "당신은 전문 여행 기획자입니다.\n\n"
            "역할:\n"
            "- 고객 관심사 반영 맞춤형 여행 계획\n"
            "- 테마별 일정 및 특별 경험 제안\n"
            "- 현지 문화 체험 기회\n"
            "- 숨겨진 명소 및 로컬 스팟\n"
            "- 부티크 호텔 및 미슐랭 레스토랑\n"
            "- 교통 최적화 및 상세 예산\n\n"
            "스타일: 전문적이고 창의적인 계획, 모든 응답은 한글로 작성"
        )
    )
    
    # Agent 4: Luxury Travel Concierge (럭셔리 여행 컨시어지)
    luxury_concierge = agent_client.create_agent(
        name="LuxuryConcierge",
        instructions=(
            "당신은 럭셔리 여행 컨시어지입니다.\n\n"
            "역할:\n"
            "- 최고급 서비스 및 독점적 경험 제공\n"
            "- VIP 전용 경험 및 프라이빗 투어\n"
            "- 5성급 호텔 또는 프라이빗 빌라\n"
            "- 미슐랭 3스타 레스토랑 예약\n"
            "- 전용 차량 및 가이드 서비스\n"
            "- 프리미엄 스파 & 웰니스\n"
            "- 24시간 컨시어지 서비스\n\n"
            "스타일: 품격 있고 세련된 제안, 모든 응답은 한글로 작성"
        )
    )
    
    # Agent 5: Coordinator (최종 정리)
    coordinator = agent_client.create_agent(
        name="Coordinator",
        instructions=(
            "당신은 여행 코디네이터입니다.\n\n"
            "역할:\n"
            "- 전문가 계획을 완성도 높은 제안서로 정리\n"
            "- 여행 계획 완결성 확인\n"
            "- 명확한 일정표 형식 제공\n"
            "- 예산 요약 및 예약 절차 안내\n"
            "- 다음 단계 및 연락처 정보\n"
            "- 추가 맞춤화 옵션 제시\n\n"
            "스타일: 체계적이고 완성도 높은 문서, 모든 응답은 한글로 작성"
        )
    )
    
    print(f"✅ Agent 1: {triage_agent.name} (Triage Agent) 생성 완료")
    print(f"✅ Agent 2: {general_advisor.name} (General Advisor) 생성 완료")
    print(f"✅ Agent 3: {specialist_planner.name} (Specialist Planner) 생성 완료")
    print(f"✅ Agent 4: {luxury_concierge.name} (Luxury Concierge) 생성 완료")
    print(f"✅ Agent 5: {coordinator.name} (Coordinator) 생성 완료")
    print("="*70 + "\n")
    
    return triage_agent, general_advisor, specialist_planner, luxury_concierge, coordinator

# Handoff Agents 생성
triage_agent, general_advisor, specialist_planner, luxury_concierge, coordinator = create_handoff_agents()

# ========================================================================
# Step 1: Executor 노드 정의 (핸드오프 패턴)
# ========================================================================

@executor(id="triage_agent")
async def triage_agent_node(msg: TravelPlanRequest, ctx: WorkflowContext[TravelPlanRequest]) -> None:
    """Step 1: 여행 요청 분석 및 복잡도 평가"""
    print(f"\n🔍 Step 1: Triage Agent - 여행 요청 분석 중...")
    print(f"📩 요청: {msg.travel_request[:100]}...")
    
    query = f"""다음 여행 요청을 분석하고 복잡도를 평가하세요:

여행 요청: {msg.travel_request}

다음 항목을 평가하세요:
1. 복잡도 점수 (0-100)
2. 카테고리 (간단함/맞춤형/럭셔리)
3. 평가 이유
4. 주요 키워드
5. 추천 전문가 타입

평가 기준:
- 0-30: 간단한 여행 (주요 도시, 짧은 기간, 일반 관광)
- 31-70: 맞춤형 여행 (테마, 여러 도시, 특정 관심사)
- 71-100: 럭셔리/복잡한 여행 (프라이빗 투어, 고급 숙소, 특별 경험)

반드시 한글로 상세히 답변하세요."""
    
    # Triage Agent 실행
    thread = triage_agent.get_new_thread()
    result = await triage_agent.run(query, thread=thread)
    response = result.text if hasattr(result, 'text') else str(result)
    
    msg.triage_analysis = response
    
    # 복잡도 점수 추출 (키워드 기반)
    if any(word in response for word in ['간단', '짧은', '주말', '일반']):
        msg.complexity_score = 20
        msg.assigned_specialist = "일반여행상담"
        target_specialist = "general_advisor"
    elif any(word in response for word in ['럭셔리', '복잡', '프라이빗', '특별', '고급', 'VIP']):
        msg.complexity_score = 85
        msg.assigned_specialist = "럭셔리컨시어지"
        target_specialist = "luxury_concierge"
    else:
        msg.complexity_score = 55
        msg.assigned_specialist = "전문여행기획자"
        target_specialist = "specialist_planner"
    
    msg.status = "analyzed"
    
    print(f"✅ 분석 완료")
    print(f"🎯 복잡도 점수: {msg.complexity_score}/100")
    print(f"🎯 배정 전문가: {msg.assigned_specialist}")
    print(f"분석 결과 미리보기:\n{response[:300]}...\n")
    
    # 적절한 전문가에게 핸드오프
    await ctx.send_message(msg, target_id=target_specialist)


@executor(id="general_advisor")
async def general_advisor_node(msg: TravelPlanRequest, ctx: WorkflowContext[TravelPlanRequest]) -> None:
    """Step 2a: 일반 여행 상담"""
    print(f"\n✈️  Step 2a: General Advisor - 기본 여행 계획 작성 중...")
    
    query = f"""다음 여행 요청에 친절하고 명확한 기본 여행 계획을 제공하세요:

요청: {msg.travel_request}

분석 결과: {msg.triage_analysis[:200]}...

다음 내용을 포함하세요:
- 추천 여행 기간 (2-3일 정도)
- 주요 관광지 3-5곳
- 숙소 추천 (중급 호텔)
- 대중교통 이용 방법
- 대략적인 예산 가이드
- 간단한 식당 추천

쉽고 실용적인 여행 계획을 한글로 작성하세요."""
    
    # General Advisor Agent 실행
    thread = general_advisor.get_new_thread()
    result = await general_advisor.run(query, thread=thread)
    response = result.text if hasattr(result, 'text') else str(result)
    
    msg.specialist_response = response
    msg.status = "specialist_completed"
    
    print(f"✅ 기본 여행 계획 완료")
    print(f"계획 미리보기:\n{response[:300]}...\n")
    
    # Coordinator로 전달
    await ctx.send_message(msg, target_id="coordinator")


@executor(id="specialist_planner")
async def specialist_planner_node(msg: TravelPlanRequest, ctx: WorkflowContext[TravelPlanRequest]) -> None:
    """Step 2b: 전문 여행 기획"""
    print(f"\n🎨 Step 2b: Specialist Planner - 맞춤형 여행 계획 작성 중...")
    
    query = f"""다음 여행 요청에 맞춤형 상세 여행 계획을 작성하세요:

요청: {msg.travel_request}

분석 결과: {msg.triage_analysis[:200]}...

다음 내용을 포함하세요:
- 테마별 맞춤 일정 (일별 상세 계획)
- 특별한 경험 및 액티비티 추천
- 현지 문화 체험 기회
- 숨겨진 명소 및 로컬 스팟
- 맞춤형 숙소 옵션 (부티크 호텔, 게스트하우스 등)
- 식도락 투어 (미슐랭, 로컬 맛집)
- 교통 및 이동 최적화
- 예산 상세 breakdown

전문적이고 창의적인 여행 계획을 한글로 작성하세요."""
    
    # Specialist Planner Agent 실행
    thread = specialist_planner.get_new_thread()
    result = await specialist_planner.run(query, thread=thread)
    response = result.text if hasattr(result, 'text') else str(result)
    
    msg.specialist_response = response
    msg.status = "specialist_completed"
    
    print(f"✅ 맞춤형 여행 계획 완료")
    print(f"계획 미리보기:\n{response[:300]}...\n")
    
    # Coordinator로 전달
    await ctx.send_message(msg, target_id="coordinator")


@executor(id="luxury_concierge")
async def luxury_concierge_node(msg: TravelPlanRequest, ctx: WorkflowContext[TravelPlanRequest]) -> None:
    """Step 2c: 럭셔리 여행 컨시어지"""
    print(f"\n💎 Step 2c: Luxury Concierge - 럭셔리 여행 계획 작성 중...")
    
    query = f"""다음 여행 요청에 최고급 럭셔리 여행 계획을 작성하세요:

요청: {msg.travel_request}

분석 결과: {msg.triage_analysis[:200]}...

다음 내용을 포함하세요:
- VIP 전용 경험 및 프라이빗 투어
- 5성급 럭셔리 호텔 또는 프라이빗 빌라
- 미슐랭 3스타 레스토랑 예약
- 전용 차량 및 가이드 서비스
- 독점적인 문화/예술 경험
- 프리미엄 스파 & 웰니스
- 헬리콥터 투어 또는 프라이빗 제트 옵션
- 24시간 컨시어지 서비스
- 완벽한 일정 조율 및 예약 지원

최고급 서비스와 독특한 경험을 제공하는 럭셔리 여행 계획을 한글로 작성하세요."""
    
    # Luxury Concierge Agent 실행
    thread = luxury_concierge.get_new_thread()
    result = await luxury_concierge.run(query, thread=thread)
    response = result.text if hasattr(result, 'text') else str(result)
    
    msg.specialist_response = response
    msg.status = "specialist_completed"
    
    print(f"✅ 럭셔리 여행 계획 완료")
    print(f"계획 미리보기:\n{response[:300]}...\n")
    
    # Coordinator로 전달
    await ctx.send_message(msg, target_id="coordinator")


@executor(id="coordinator")
async def coordinator_node(msg: TravelPlanRequest, ctx: WorkflowContext[TravelPlanRequest]) -> None:
    """Step 3: 최종 여행 제안서 정리"""
    print(f"\n📝 Step 3: Coordinator - 최종 여행 제안서 작성 중...")
    
    query = f"""다음 전문가 여행 계획을 검토하고 최종 고객 제안서를 작성하세요:

원래 요청: {msg.travel_request}
복잡도: {msg.complexity_score}/100
담당 전문가: {msg.assigned_specialist}

전문가 계획:
{msg.specialist_response[:800]}...

최종 제안서 작성 가이드:
1. 여행 계획의 완결성 확인
2. 예약 가능 여부 및 실현 가능성 점검
3. 명확한 일정표 형식으로 정리
4. 예산 요약 추가
5. 예약 절차 및 다음 단계 안내
6. 추가 맞춤화 옵션 제시
7. 연락처 및 지원 정보

고객에게 전달할 완성된 여행 제안서를 한글로 작성하세요."""
    
    # Coordinator Agent 실행
    thread = coordinator.get_new_thread()
    result = await coordinator.run(query, thread=thread)
    response = result.text if hasattr(result, 'text') else str(result)
    
    msg.final_proposal = response
    msg.status = "completed"
    
    print(f"✅ 최종 제안서 완성")
    
    await ctx.yield_output(msg)


# ========================================================================
# Step 2: WorkflowBuilder로 핸드오프 그래프 구성
# ========================================================================

handoff_workflow = (
    WorkflowBuilder()
    # Triage Agent에서 시작
    .set_start_executor(triage_agent_node)
    # Triage → 3개 전문가 (동적 라우팅)
    .add_edge(triage_agent_node, general_advisor_node)
    .add_edge(triage_agent_node, specialist_planner_node)
    .add_edge(triage_agent_node, luxury_concierge_node)
    # 각 전문가 → Coordinator
    .add_edge(general_advisor_node, coordinator_node)
    .add_edge(specialist_planner_node, coordinator_node)
    .add_edge(luxury_concierge_node, coordinator_node)
    .build()
)

print("\n" + "="*70)
print("✅ Handoff Workflow 그래프 구성 완료")
print("="*70)
print("📊 Workflow 구조 (핸드오프 패턴):")
print("   triage_agent_node (복잡도 평가)")
print("         ↓")
print("   [동적 라우팅]")
print("    ├─→ general_advisor_node (간단한 여행)")
print("    ├─→ specialist_planner_node (맞춤형 여행)")
print("    └─→ luxury_concierge_node (럭셔리 여행)")
print("         ↓")
print("   coordinator_node (최종 정리)")
print("="*70 + "\n")


# ========================================================================
# Step 3: Workflow 실행
# ========================================================================

async def run_handoff_workflow(travel_request: str):
    """Handoff Workflow 실행 함수"""
    print("\n" + "="*70)
    print("🚀 Handoff Workflow 실행 시작 (MAF WorkflowBuilder)")
    print("="*70)
    print(f"📩 여행 요청: {travel_request[:100]}...")
    print("="*70)

    # 입력 메시지 생성
    plan_request = TravelPlanRequest(
        travel_request=travel_request
    )

    # Workflow 실행 (run_stream 사용 - async generator)
    outputs = []
    async for event in handoff_workflow.run_stream(plan_request):
        # 이벤트에서 output 추출
        if hasattr(event, 'output') and event.output is not None:
            outputs.append(event.output)
            print(f"📤 이벤트 수신: {type(event.output).__name__}")
        elif hasattr(event, 'data') and event.data is not None:
            outputs.append(event.data)
            print(f"📤 이벤트 수신: {type(event.data).__name__}")

    # 최종 결과 추출
    if outputs:
        final_result = outputs[-1]
    else:
        raise ValueError("❌ Workflow에서 출력을 받지 못했습니다.")

    print("\n" + "="*70)
    print("📊 Handoff Pattern 결과 (MAF WorkflowBuilder)")
    print("="*70)
    print(f"\n📝 여행 요청: {final_result.travel_request[:100]}...")
    print(f"✅ 복잡도 점수: {final_result.complexity_score}/100")
    print(f"✅ 배정 전문가: {final_result.assigned_specialist}")
    print(f"✅ 최종 상태: {final_result.status}")
    
    print(f"\n{'='*70}")
    print(f"🔍 복잡도 분석:")
    print(f"{'='*70}")
    print(f"{final_result.triage_analysis}\n")
    
    print(f"\n{'='*70}")
    print(f"💬 전문가 응답:")
    print(f"{'='*70}")
    print(f"{final_result.specialist_response}\n")
    
    print(f"\n{'='*70}")
    print(f"✨ 최종 여행 제안서:")
    print(f"{'='*70}")
    print(f"{final_result.final_proposal}")
    
    print(f"\n{'='*70}")
    
    return final_result

# Handoff Workflow 실행 - 다양한 복잡도 테스트
print("\n\n🧪 시나리오 1: 간단한 여행 - 일반 여행 상담 에이전트로 핸드오프")
result1 = await run_handoff_workflow(
    "주말에 부산 2박 3일 여행 계획을 짜주세요. 주요 관광지와 맛집 추천 부탁드립니다."
)

# 시나리오 2: 맞춤형 여행 (전문 여행 기획자)
print("\n\n🧪 시나리오 2: 맞춤형 여행 - 전문 여행 기획자로 핸드오프")
result2 = await run_handoff_workflow(
    "일본 교토에서 전통 문화와 선(禪) 명상을 중심으로 한 5일간의 힐링 여행을 계획하고 싶습니다. 사찰 스테이, 다도 체험, 정원 투어를 포함한 깊이 있는 문화 체험을 원합니다."
)

# 시나리오 3: 럭셔리 여행 (럭셔리 여행 컨시어지)
print("\n\n🧪 시나리오 3: 럭셔리 여행 - 럭셔리 여행 컨시어지로 핸드오프")
result3 = await run_handoff_workflow(
    "가족 4명이 함께하는 2주간의 유럽 럭셔리 투어를 원합니다. 파리, 밀라노, 산토리니를 방문하며, 미슐랭 3스타 레스토랑, 5성급 호텔, 프라이빗 가이드 투어, 그리고 특별한 VIP 경험을 포함해주세요. 예산 제한은 없으며 최고의 경험을 원합니다."
)

### Handoff Pattern 분석 및 정리

#### 📊 핸드오프 패턴의 장점

1. **적응적 전문성 활용**
   - 여행 복잡도에 따라 적절한 수준의 전문가 배정
   - 리소스 효율적 사용 (과도한 전문가 투입 방지)
   - 각 에이전트가 자신의 전문 분야에 집중

2. **확장 가능한 구조**
   - 새로운 전문가 에이전트를 쉽게 추가 가능
   - 복잡도 기준을 조정하여 유연한 라우팅
   - 다양한 도메인으로 확장 가능

3. **품질 보장**
   - Triage Agent가 정확한 분류 수행
   - 각 전문가가 최적화된 여행 계획 제공
   - Coordinator가 최종 품질 검증

4. **추적 및 모니터링**
   - 핸드오프 이력을 통한 패턴 분석
   - 병목 구간 식별 및 개선
   - 에이전트별 성능 측정

#### 🎯 실제 적용 사례

**여행 계획 서비스**
```
Level 1 → 간단한 여행 (일반 상담원 - 패키지 여행)
Level 2 → 맞춤형 여행 (전문 기획자 - 테마 여행)
Level 3 → 럭셔리 여행 (컨시어지 - VIP 경험)
Level 4 → 초프리미엄 (개인 전담 - 완전 맞춤)
```

**고객 지원 센터**
```
Level 1 → 일반 FAQ (자동 응답)
Level 2 → 기술 지원 (전문 상담원)
Level 3 → 고급 엔지니어 (복잡한 문제)
Level 4 → 매니저 승인 (정책 변경 필요)
```

**의료 시스템**
```
증상 체크 → 간호사 (일반 상담)
          → 일반의 (진단 필요)
          → 전문의 (특수 치료)
          → 수술팀 (수술 필요)
```

#### ⚙️ 구현 고려사항

1. **명확한 복잡도 기준 설정**
   - 정량적 기준 (키워드, 요청 길이, 예산 등)
   - 정성적 기준 (도메인 특화 규칙)
   - 학습 데이터 기반 ML 모델 활용 가능

2. **컨텍스트 전달 최적화**
   - 필수 정보만 전달 (과도한 컨텍스트 방지)
   - 구조화된 형식 사용 (JSON 등)
   - 핸드오프 이유 명시

3. **에스컬레이션 정책**
   - 자동 에스컬레이션 조건 정의
   - 수동 에스컬레이션 요청 지원
   - 에스컬레이션 이력 추적

4. **성능 최적화**
   - 캐싱을 통한 반복 분석 방지
   - 비동기 처리로 대기 시간 감소
   - 병렬 처리 가능한 부분 식별

#### 🔄 다른 패턴과의 조합

**Handoff + Sequential**
```python
# 순차적으로 여러 전문가를 거치는 워크플로우
triage → travel_planner → budget_optimizer → booking_agent
```

**Handoff + Parallel**
```python
# 여러 전문가에게 동시에 자문 후 통합
triage → [flight_expert, hotel_expert, activity_expert] → coordinator
```

**Handoff + Loop**
```python
# 만족할 때까지 반복 개선
triage → specialist → customer_review → (불만족) → specialist (다시)
                                      → (만족) → 완료
```

---

## 📊 워크플로우 패턴 종합 비교

### 패턴별 특징 요약 (실습 순서)

| 실습 | 패턴 | 실행 방식 | 시간 복잡도 | 주요 사용 사례 | 핵심 장점 | 주의사항 |
|------|------|---------|------------|--------------|----------|---------|
| **2번** | **Sequential** | 순차 실행 | T1 + T2 + T3 | 여행 일정 작성 → 현지 전문가 리뷰 → 최종 정리 | 명확한 흐름, 구현 간단 | 느린 실행 속도 |
| **3번** | **Concurrent** | 병렬 실행 | max(T1, T2, T3) | 여러 도시 동시 분석, 다각적 리뷰 | 빠른 처리, 다양한 관점 | 동기화 관리 필요 |
| **4번** | **Conditional** | 조건 분기 | Classifier + Expert | 여행 스타일별 전문가 라우팅 | 맞춤형 응답, 효율적 처리 | 분류 정확도 중요 |
| **5번** | **Loop** | 반복 개선 | N × T | 여행 계획 반복 개선, 품질 향상 | 점진적 개선, 피드백 반영 | 무한 루프 방지 필요 |
| **6번** | **Error Handling** | 재시도 + 폴백 | Variable | 예약 실패 시 대안 제시 | 안정성, 복원력, 자동 복구 | 추가 오버헤드 |
| **7번** | **Handoff** | 동적 라우팅 | Triage + Specialist | 복잡도별 전문가 배정 | 적응적 전문성, 리소스 최적화 | 복잡도 평가 기준 설정 |

### 패턴 선택 의사결정 트리

```
여행 계획 작업 분석:
│
├─ 순차적 단계가 명확한가?
│  ├─ Yes → Sequential Pattern (2번 실습)
│  │         (예: 일정 작성 → 검증 → 예약)
│  └─ No → 다음으로
│
├─ 독립적인 여러 관점이 필요한가?
│  ├─ Yes → Concurrent Pattern (3번 실습)
│  │         (예: 항공, 숙소, 액티비티 동시 검색)
│  └─ No → 다음으로
│
├─ 입력 조건에 따라 다른 전문가가 필요한가?
│  ├─ Yes → Conditional Pattern (4번 실습)
│  │         (예: 여행 스타일별 전문가 선택)
│  └─ No → 다음으로
│
├─ 반복적 개선이 필요한가?
│  ├─ Yes → Loop Pattern (5번 실습)
│  │         (예: 고객 만족도 기준 충족까지 개선)
│  └─ No → 다음으로
│
├─ 외부 서비스 실패에 대비해야 하는가?
│  ├─ Yes → Error Handling Pattern (6번 실습)
│  │         (예: 예약 실패 시 자동 복구)
│  └─ No → 다음으로
│
└─ 복잡도에 따른 차별화가 필요한가?
   └─ Yes → Handoff Pattern (7번 실습)
             (예: 간단한 여행 vs 럭셔리 여행)
```

### 실전 조합 패턴

실제 프로젝트에서는 여러 패턴을 조합하여 사용:

**조합 1: Sequential + Error Handling (2번 + 6번)**
```
일정 작성 → 예약 시도 → (실패 시) 대안 제시 → 재예약 → 확정
```
- 사용 사례: 안정적인 여행 예약 시스템

**조합 2: Concurrent + Conditional (3번 + 4번)**
```
[문화 전문가, 음식 전문가, 액티비티 전문가] → 스타일별 필터링 → 통합
```
- 사용 사례: 맞춤형 여행 추천 시스템

**조합 3: Handoff + Loop (7번 + 5번)**
```
복잡도 평가 → 전문가 배정 → 계획 작성 → 피드백 → (개선) 반복 → 완료
```
- 사용 사례: 프리미엄 여행 컨설팅

**조합 4: Handoff + Sequential (7번 + 2번)**
```
복잡도 평가 → 전문가 선택 → 일정 작성 → 검증 → 확정
```
- 사용 사례: 일반 여행 계획 서비스

### 실습별 핵심 개념 정리

**2번 - Sequential Pattern (순차 패턴)**
- 개념: 3개 agent가 순차적으로 작업 수행
- 구조: planner → reviewer → finalizer
- 학습 포인트: `add_edge()`로 순차 연결, 각 단계별 결과 누적

**3번 - Concurrent Pattern (병렬 패턴)**
- 개념: 여러 agent가 동시 작업 후 통합
- 구조: broadcast → [culture, food, practical] → aggregator
- 학습 포인트: `broadcast_start_node`로 팬아웃, 여러 경로 병렬 실행

**4번 - Conditional Pattern (조건 분기 패턴)**
- 개념: 입력에 따라 다른 전문가 선택
- 구조: classifier → [culture/activity/relaxation/gourmet expert]
- 학습 포인트: `target_id` 동적 지정으로 조건부 라우팅

**5번 - Loop Pattern (반복 패턴)**
- 개념: 조건 만족까지 반복 개선
- 구조: generator ↔ analyzer (최대 3회) → optimizer
- 학습 포인트: `iteration_count` 추적, 조건부 순환/종료

**6번 - Error Handling Pattern (에러 처리 패턴)**
- 개념: 실패 감지 → 분석 → 대안 → 복구
- 구조: booking → error_analyzer → fallback_planner → recovery
- 학습 포인트: 키워드 기반 에러 감지, 자동 대안 생성

**7번 - Handoff Pattern (핸드오프 패턴)**
- 개념: 복잡도 평가 후 적절한 전문가에게 위임
- 구조: triage → [general/specialist/luxury advisor] → coordinator
- 학습 포인트: 동적 라우팅, 복잡도 기반 전문가 배정

---

## 🎯 Azure AI Foundry Agent 베스트 프랙티스

### 1. 프롬프트 작성 원칙

#### ✅ 좋은 프롬프트 예시
```python
prompt = """
작업: 파리 5일 여행 일정 작성

요구사항:
- 기간: 2025년 11월 15일 ~ 11월 19일
- 관심사: 미술관, 카페 문화, 현지 음식
- 예산: 중상급

출력 형식:
- 일별 상세 일정 (시간대별)
- 추천 레스토랑 3곳
- 교통 정보
- 예상 비용

한글로 작성하세요.
"""
```

#### ❌ 나쁜 프롬프트 예시
```python
prompt = "파리 여행 계획 좀 짜줘"  # 너무 모호함
```

### 2. 에이전트 역할 정의

각 에이전트에게 **명확한 역할과 전문성**을 부여:

```python
# Travel Planner Agent
instructions = """
당신은 전문 여행 기획자입니다.

전문 분야:
- 일정 최적화
- 현지 문화 이해
- 예산 관리
- 숨겨진 명소 추천

작업 방식:
1. 고객 요구사항 정확히 파악
2. 실현 가능한 일정 작성
3. 대안 옵션 제시
4. 명확한 근거와 함께 설명
"""
```

### 3. 오류 처리 전략

```python
# ✅ 체계적인 오류 처리
try:
    result = await call_foundry_agent(query, instructions)
    
    # 결과 검증
    if not result or len(result) < 50:
        raise ValueError("응답이 너무 짧습니다")
        
except Exception as e:
    logger.error(f"에이전트 호출 실패: {e}")
    
    # 폴백 전략
    fallback_result = await call_foundry_agent(
        simplified_query,
        fallback_instructions
    )
```

### 4. 성능 최적화

#### 병렬 처리 활용
```python
# ✅ 독립적인 작업은 병렬로
results = await asyncio.gather(
    call_foundry_agent(query1, instructions1),
    call_foundry_agent(query2, instructions2),
    call_foundry_agent(query3, instructions3)
)
```

#### 토큰 사용량 최적화
- ✅ 필수 정보만 컨텍스트에 포함
- ✅ 이전 대화는 요약하여 전달
- ✅ 불필요한 반복 설명 제거

### 5. 실전 체크리스트

**에이전트 호출 전:**
- [ ] 프롬프트가 명확하고 구체적인가?
- [ ] 출력 형식이 정의되어 있는가?
- [ ] 에이전트 역할이 명확한가?
- [ ] 예상 토큰 사용량이 적절한가?

**에이전트 호출 후:**
- [ ] 응답이 요구사항을 충족하는가?
- [ ] 오류 처리가 되어 있는가?
- [ ] 실행 시간이 허용 범위 내인가?
- [ ] 로그가 적절히 기록되었는가?

### 6. 주요 주의사항

⚠️ **무한 루프 방지**
```python
max_iterations = 10  # 최대 반복 횟수 설정
for i in range(max_iterations):
    result = await process()
    if condition_met:
        break
```

⚠️ **타임아웃 설정**
```python
import asyncio

try:
    result = await asyncio.wait_for(
        call_foundry_agent(query, instructions),
        timeout=30.0  # 30초 타임아웃
    )
except asyncio.TimeoutError:
    logger.error("요청 타임아웃")
```

⚠️ **리소스 정리**
```python
# 생성한 에이전트와 스레드는 반드시 정리
try:
    # 작업 수행
    result = await process()
finally:
    # 리소스 정리
    if agent_id:
        foundry_client.agents.delete_agent(agent_id)
    if thread_id:
        foundry_client.agents.threads.delete(thread_id)
```

---

## 🎓 학습 완료 및 다음 단계

### 학습 완료 체크리스트

이 노트북에서 학습한 내용:

- ✅ **Sequential Pattern**: 순차적 여행 계획 작성 (플래너 → 전문가 → 요약)
- ✅ **Concurrent Pattern**: 병렬 여행 분석 (문화/모험/경치 동시 평가)
- ✅ **Loop Pattern**: 반복 개선을 통한 여행 일정 최적화
- ✅ **Handoff Pattern**: 복잡도별 전문가 동적 배정
- ✅ **Error Handling Pattern**: 예약 실패 시 자동 대안 제시
- ✅ **Azure AI Foundry Agent** 통합 및 사용법
- ✅ **비동기 프로그래밍** (`async`/`await`)
- ✅ **프롬프트 엔지니어링** 기법
- ✅ **패턴 선택 의사결정**

### 실전 적용 가이드

#### 1단계: 문제 정의
```
질문: "우리 프로젝트에 어떤 패턴이 적합한가?"

체크포인트:
- 작업 간 의존성이 있는가? → Sequential
- 독립적인 여러 분석이 필요한가? → Concurrent
- 조건에 따라 다른 처리가 필요한가? → Handoff
- 품질 개선을 위한 반복이 필요한가? → Loop
- 외부 서비스 의존성이 높은가? → Error Handling
```

#### 2단계: 프로토타입 구현
```python
# 가장 간단한 패턴부터 시작
# 1. Sequential로 기본 플로우 구현
# 2. 병목 구간 확인
# 3. 필요한 곳에 다른 패턴 적용
```

#### 3단계: 테스트 및 최적화
- 다양한 입력으로 테스트
- 실행 시간 및 비용 측정
- 오류 케이스 처리 확인

#### 4단계: 프로덕션 배포
- 환경 변수 설정
- 모니터링 구성
- 로깅 및 알림 설정

### 유용한 리소스

#### 공식 문서
- 🔗 [Azure AI Foundry 문서](https://learn.microsoft.com/azure/ai-studio/)
- 🔗 [Azure AI Agent Service](https://learn.microsoft.com/azure/ai-services/agents/)
- 🔗 [Python asyncio 가이드](https://docs.python.org/3/library/asyncio.html)

#### 추가 학습 자료
- 📖 본 리포지토리의 다른 노트북 (01-04)
- 📖 `OBSERVABILITY.md` - 관찰성 및 모니터링
- 📖 `MODEL_CHANGE_GUIDE.md` - 모델 변경 가이드

### 문제 해결 (Troubleshooting)

**일반적인 문제와 해결책:**

| 문제 | 원인 | 해결책 |
|------|------|--------|
| `AIProjectClient` 연결 실패 | 환경 변수 미설정 | `AZURE_AI_PROJECT_ENDPOINT` 확인 |
| 타임아웃 오류 | 프롬프트가 너무 복잡 | 작업 단위 축소, 타임아웃 증가 |
| 높은 토큰 사용량 | 불필요한 컨텍스트 | 프롬프트 최적화, 요약 사용 |
| 일관성 없는 결과 | 모호한 지시사항 | 구체적인 프롬프트 작성 |
| 에이전트 삭제 실패 | 리소스 정리 누락 | `finally` 블록에서 정리 |

### 피드백 및 기여

이 Lab이 도움이 되셨나요? 

- 🐛 버그 리포트: GitHub Issues
- 💡 개선 제안: Pull Request
- 💬 질문: GitHub Discussions

---

## 🎉 축하합니다!

**Azure AI Foundry Agent를 활용한 워크플로우 패턴 학습을 완료하셨습니다!**

이제 여러분은:
- ✨ 5가지 핵심 워크플로우 패턴을 이해하고 구현할 수 있습니다
- ✨ 복잡한 AI 에이전트 시스템을 설계할 수 있습니다
- ✨ 실전 프로젝트에 패턴을 적용할 수 있습니다
- ✨ 프로덕션 환경에 배포할 준비가 되었습니다

**다음 단계로 나아가세요!** 🚀

여러분의 AI 에이전트 개발 여정에 행운을 빕니다! 💪